# Data Reading and Data Preparation Script


In this Script


1- Data Reading 

2- Data Preparation to main dataset

### Change Log - Sept 21st

#### Libraries

In [2]:
#Test
#Libraries
import warnings
warnings.filterwarnings('ignore')

import pandas as pd 
import numpy as np
#from pycaret.time_series import *
from functools import reduce
import sys
import openpyxl
import matplotlib.pyplot as plt
import numpy
import matplotlib.pyplot as plt
from datetime import date
import pyarrow.csv as csv

plt.rcParams['figure.figsize'] = [20, 10]
from pandas import read_csv
import math


from numpy import sqrt 

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error


import numpy as np

import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns

from sklearn.preprocessing import StandardScaler





# Data Sourcing

In [3]:
# GCP DS bucket (latest - coming from auto script)

#1
main_orders = pd.read_csv("gs://flextock_ds_buuket/Demand Forecasting/TestData/metabase_outbounds.csv")

#2
outbounds = pd.read_csv("gs://flextock_ds_buuket/Demand Forecasting/TestData/merchant_data.csv")

#3
# alkaseeba = pd.read_csv("gs://flextock_ds_buuket/Demand Forecasting/TestData/Kaseeba_orders.csv", engine = "pyarrow")
alkaseeba = pd.read_csv("gs://flextock_ds_buuket/Demand Forecasting/TestData/Kaseeba_orders.csv")

#4
rtos = pd.read_csv("gs://flextock_ds_buuket/Demand Forecasting/TestData/RTOs.csv", engine = "pyarrow")

#5
##
inventory=pd.read_csv("gs://flextock_ds_buuket/Demand Forecasting/TestData/inventory_sku.csv", engine = "pyarrow")

#6
inventory_mapping=pd.read_csv("gs://flextock_ds_buuket/Demand Forecasting/TestData/inventory_mapping.csv", engine = "pyarrow")

#7
##
enas_sheet=pd.read_csv("gs://flextock_ds_buuket/Demand Forecasting/TestData/enas_sheet.csv", engine = "pyarrow")
# enas_sheet=pd.read_excel("gs://flextock_ds_buuket/Demand Forecasting/Files/July 24/Enas Sheet.xlsx")

#8
##
manual_input=pd.read_csv("gs://flextock_ds_buuket/Demand Forecasting/TestData/Manual_Input.csv", engine = "pyarrow")
# manual_input=pd.read_excel("gs://flextock_ds_buuket/Demand Forecasting/Files/Manual Data Input.xlsx",header =3 ,usecols="B:G")

#9
# dynamic_skus = pd.read_csv("gs://flextock_ds_buuket/Demand Forecasting/TestData/dynamic_inventory.csv", engine = "pyarrow")
dynamic_skus = pd.read_csv("gs://flextock_ds_buuket/Demand Forecasting/TestData/dynamic_inventory_all.csv", engine = "pyarrow")


#10
kaseeba_internal_seller_egypt = pd.read_csv("gs://flextock_ds_buuket/Demand Forecasting/TestData/Kaseeba_Internal_Sellers_Egypt.csv", engine = "pyarrow")

#11
prod_cat_subcat = pd.read_csv("gs://flextock_ds_buuket/Demand Forecasting/TestData/Product_cat_subcat.csv", engine = "pyarrow")


In [4]:
alkaseeba_for_confirmation_rate = alkaseeba.copy()

In [5]:
alkaseeba_active_seller= alkaseeba.copy()
# alkaseeba_active_seller.head(1)

In [6]:
# dates_0 = pd.DataFrame(pd.date_range("12-01-2021", periods=257, freq="D"))
dates_0 = pd.DataFrame(pd.date_range(start = date.today() - pd.DateOffset(730), end = date.today() - pd.DateOffset(1), freq="D"))
dates_0.columns = ['Join_Dates']
dates_0.head()

,Join_Dates
0,2021-02-08
1,2021-02-09
2,2021-02-10
3,2021-02-11
4,2021-02-12


# Data Preparation

In [7]:
pd.set_option('display.max_columns', None)
main_orders.tail(1)
# main_orders.groupby(['Merchant'],as_index=False).mean()#.pivot(columns=['Uniform_Date']User ID, values=['Items Cost'])

,order_id,merchant,order_code,total_quantity,items_cost,last_status,created_at,fulfilled_on,delivered_on,returned_to_origin_on,created_date,fulfilled_date,delivered_date,returned_to_origin_date,city,user_id,order_type,customer_name,phone_number,area,cash_returned,courier_name,tracking_number,country,location_id,invoice_id,zone_name,mapped_area,mapped_city,pickup_date,valid_phone,valid_address_line1,valid_area,valid_city,valid_country,in_stock,merchant_type,industry,ecommerce_type,weight,is_kit,sub_status
1106384,0e4097da-c4f2-4be8-b1ad-7ca9cc8a3dff,Zvedno,ZVN-OT-773,2,120.0,pending,2022-03-14 13:57:04.259203+00:00,NaN,NaN,NaN,2022-03-14,NaN,NaN,NaN,Cairo,1374,outbound_order,NaN,00201254782589,1st Settlement,0,NaN,NaN,Egypt,NaN,NaN,cairo_giza,NaN,Cairo,NaN,True,False,False,True,True,False,Brand,Electronics,NaN,NaN,NaN,NaN


In [8]:
main_orders.shape

(1106385, 42)

In [9]:
main_orders.order_id.nunique()

1106385

In [10]:
main_orders.order_code.nunique()

1106385

In [11]:
main_orders.order_type.unique()

array(['outbound_order'], dtype=object)

In [12]:
main_orders.last_status.unique()

array(['delivered', 'returned to origin', 'canceled', 'processing',
       'pick up', 'on hold', 'lost', 'in transit', 'fulfilled', 'pending',
       'ready'], dtype=object)

In [13]:
main_orders['created_at'].max()

'2023-02-07 23:42:41.687790+00:00'

In [14]:
# pd.set_option('display.max_columns', None)
outbounds.head(1)

,address_line1,address_line2,area,building_no,city,created_at,first_name,floor_no,is_work_address,items_cost,last_name,merchant_name,modified_at,note,order_code,order_customer_name,order_date,phone_number,secondary_phone_number,sku_code,sku_name,sku_pieces,status,user_id,username,week_no,week_number,tracking_number
0,Address: الغربيه طنطا مركز قطور امام صدناوي شا...,NaN,Tanta,0,Gharbia,2021-06-20 14:32:54.086180+00:00,Salma Adel,NaN,False,465.0,.,Sara (rhea),2021-06-21 17:26:21.938044+00:00,NaN,RHE-OT-909991,Salma Adel .,2021-06-20,01022659816,NaN,K1,KERACELL SERUM,1,delivered,1057,rhea,2021-25,2021-25,46706446290


In [15]:
outbounds.shape

(1537610, 28)

In [16]:
outbounds.order_code.nunique()

1009426

In [17]:
outbounds.status.unique()

array(['delivered', 'returned to origin', 'in transit', 'pending',
       'on hold', 'processing', 'lost', 'ready', 'pick up', 'fulfilled'],
      dtype=object)

In [18]:
outbounds.order_code.nunique()

1009426

In [19]:
alkaseeba.head(1)

,Order Code,Seller,Seller Mobile,Customer,Customer Mobile,Created At,SKU,Product Name,Quantity,Status,Price,Profit,Customer Address,In Stock,Customer Notes
0,129915,Mohamed essayed reda,1200010934,متولي عطا الله عبد المطلب,1001294859,2021-12-13 13:56:16.959453+00:00,MOB-NK,موبايل عفروتو - Mini Nokia 3310,1,delivered,390.0,135.0,عزبه حامد 2 بجوار خير زمان,out of stock,NaN


In [20]:
alkaseeba['Created At'].max()

'2023-02-08 00:07:30.761501+00:00'

In [21]:
enas_sheet.columns = enas_sheet.columns.str.strip()

In [22]:
enas_sheet.shape

(852, 20)

In [23]:
alkaseeba.rename(columns = {'Order Code':'order_code'}, inplace = True)
# alkaseeba.rename(columns = {'Order Code':'order_code'}, inplace = True)
# rtos.rename(columns = {'Order Code':'order_code'}, inplace = True)


In [24]:
main_orders.head()

,order_id,merchant,order_code,total_quantity,items_cost,last_status,created_at,fulfilled_on,delivered_on,returned_to_origin_on,created_date,fulfilled_date,delivered_date,returned_to_origin_date,city,user_id,order_type,customer_name,phone_number,area,cash_returned,courier_name,tracking_number,country,location_id,invoice_id,zone_name,mapped_area,mapped_city,pickup_date,valid_phone,valid_address_line1,valid_area,valid_city,valid_country,in_stock,merchant_type,industry,ecommerce_type,weight,is_kit,sub_status
0,1c37a317-1497-4589-9f6f-aa78b2e489dd,21stiches,21S-OT-ST1001,1,550.0,delivered,2021-03-15 13:58:49.372226+00:00,2021-03-15 16:26:00+00:00,2021-03-17 17:39:28+00:00,NaN,2021-03-15,2021-03-15,2021-03-17,NaN,Cairo,1039,outbound_order,Dalia Ezzat,01225109910,Heliopolis,0,Bosta,6563100,Egypt,99.0,11796.0,cairo_giza,Heliopolis,Cairo,NaN,True,True,True,True,True,NaN,Brand,Fashion,NaN,2.0,NaN,NaN
1,78beb060-8507-4477-8286-14efb5a87b59,21stiches,21S-OT-ST1002,1,550.0,returned to origin,2021-03-15 13:58:49.817204+00:00,2021-03-15 16:29:00+00:00,NaN,2021-03-20 14:36:15.116000+00:00,2021-03-15,2021-03-15,NaN,2021-03-20,Cairo,1039,outbound_order,Nadia Khaled,01015515382,Dokki,0,Bosta,8921541,Egypt,110.0,11796.0,cairo_giza,Sheraton,Cairo,NaN,True,True,True,True,True,NaN,Brand,Fashion,NaN,2.0,NaN,NaN
2,ffaf14cc-abdf-4edb-a6e9-7208e5529d6c,21stiches,21S-OT-ST1003,1,550.0,delivered,2021-03-15 13:58:50.147332+00:00,2021-03-15 16:30:00+00:00,2021-03-16 17:24:30+00:00,NaN,2021-03-15,2021-03-15,2021-03-16,NaN,Alexandria,1039,outbound_order,Rodiana Islam,01200133524,Cleopatra,0,Bosta,609508,Egypt,NaN,11796.0,alex,NaN,Alexandria,NaN,True,True,False,True,True,NaN,Brand,Fashion,NaN,2.0,NaN,NaN
3,5cf61837-496e-41a2-bd2f-b2b74f37885b,21stiches,21S-OT-ST1004,1,550.0,delivered,2021-03-15 13:58:50.566848+00:00,2021-03-15 16:27:00+00:00,2021-03-16 19:36:55+00:00,NaN,2021-03-15,2021-03-15,2021-03-16,NaN,Cairo,1039,outbound_order,Chloe Rockingham,01276642289,Maadi,0,Bosta,2333402,Egypt,101.0,11796.0,cairo_giza,Maadi,Cairo,NaN,True,True,True,True,True,NaN,Brand,Fashion,NaN,2.0,NaN,NaN
4,ba81b4f7-a073-4ebb-b806-e0d1781ad099,21stiches,21S-OT-ST1005,1,550.0,returned to origin,2021-03-15 13:58:50.896758+00:00,2021-03-15 16:25:00+00:00,NaN,2021-03-20 14:35:57.754000+00:00,2021-03-15,2021-03-15,NaN,2021-03-20,Cairo,1039,outbound_order,Mariam Khan,01113021896,Pyramids,0,Bosta,5854069,Egypt,NaN,11796.0,cairo_giza,NaN,Cairo,NaN,True,True,False,True,True,NaN,Brand,Fashion,NaN,2.0,NaN,NaN


In [25]:
main_orders.shape

(1106385, 42)

In [26]:
#new files order_id is chnages to Order ID
main_orders.rename(columns = {'Order ID':'order_id','Order Code':'order_code'}, inplace = True)


#dont drop diplicates by metabase order_code but drop by order_ids
main_orders = main_orders.drop_duplicates(subset=['order_id'])

In [27]:
main_orders.shape

(1106385, 42)

In [28]:
main_orders.columns

Index(['order_id', 'merchant', 'order_code', 'total_quantity', 'items_cost',
       'last_status', 'created_at', 'fulfilled_on', 'delivered_on',
       'returned_to_origin_on', 'created_date', 'fulfilled_date',
       'delivered_date', 'returned_to_origin_date', 'city', 'user_id',
       'order_type', 'customer_name', 'phone_number', 'area', 'cash_returned',
       'courier_name', 'tracking_number', 'country', 'location_id',
       'invoice_id', 'zone_name', 'mapped_area', 'mapped_city', 'pickup_date',
       'valid_phone', 'valid_address_line1', 'valid_area', 'valid_city',
       'valid_country', 'in_stock', 'merchant_type', 'industry',
       'ecommerce_type', 'weight', 'is_kit', 'sub_status'],
      dtype='object')

In [29]:
alkaseeba.columns

Index(['order_code', 'Seller', 'Seller Mobile', 'Customer', 'Customer Mobile',
       'Created At', 'SKU', 'Product Name', 'Quantity', 'Status', 'Price',
       'Profit', 'Customer Address', 'In Stock', 'Customer Notes'],
      dtype='object')

In [30]:
alkaseeba.shape

(494560, 15)

In [31]:
#Kaseeba Clean ie. remove duplicates
alkaseeba = alkaseeba.drop_duplicates()
# alkaseeba = alkaseeba.drop(axis=1,columns=['Order Code'])

In [32]:
alkaseeba.shape

(494461, 15)

In [33]:
# outbounds['order_date'].dt.month
# outbounds.dtypes
outbounds.shape, main_orders.shape

((1537610, 28), (1106385, 42))

In [34]:
# data_frames = [outbounds,alkaseeba]
data_frames = [outbounds,main_orders]
df_merged = reduce(lambda  left,right: pd.merge(left,right, on =['order_code'], how='left',suffixes = ("","_main")), data_frames)


In [35]:
df_merged.last_status.unique()

array(['delivered', 'returned to origin', 'in transit', 'pending',
       'on hold', 'processing', 'lost', 'ready', 'pick up', 'fulfilled',
       nan], dtype=object)

In [36]:
df_merged.shape

(1537610, 69)

In [37]:
df_merged.columns

Index(['address_line1', 'address_line2', 'area', 'building_no', 'city',
       'created_at', 'first_name', 'floor_no', 'is_work_address', 'items_cost',
       'last_name', 'merchant_name', 'modified_at', 'note', 'order_code',
       'order_customer_name', 'order_date', 'phone_number',
       'secondary_phone_number', 'sku_code', 'sku_name', 'sku_pieces',
       'status', 'user_id', 'username', 'week_no', 'week_number',
       'tracking_number', 'order_id', 'merchant', 'total_quantity',
       'items_cost_main', 'last_status', 'created_at_main', 'fulfilled_on',
       'delivered_on', 'returned_to_origin_on', 'created_date',
       'fulfilled_date', 'delivered_date', 'returned_to_origin_date',
       'city_main', 'user_id_main', 'order_type', 'customer_name',
       'phone_number_main', 'area_main', 'cash_returned', 'courier_name',
       'tracking_number_main', 'country', 'location_id', 'invoice_id',
       'zone_name', 'mapped_area', 'mapped_city', 'pickup_date', 'valid_phone',
       

In [38]:
df_merged.tail(5)

,address_line1,address_line2,area,building_no,city,created_at,first_name,floor_no,is_work_address,items_cost,last_name,merchant_name,modified_at,note,order_code,order_customer_name,order_date,phone_number,secondary_phone_number,sku_code,sku_name,sku_pieces,status,user_id,username,week_no,week_number,tracking_number,order_id,merchant,total_quantity,items_cost_main,last_status,created_at_main,fulfilled_on,delivered_on,returned_to_origin_on,created_date,fulfilled_date,delivered_date,returned_to_origin_date,city_main,user_id_main,order_type,customer_name,phone_number_main,area_main,cash_returned,courier_name,tracking_number_main,country,location_id,invoice_id,zone_name,mapped_area,mapped_city,pickup_date,valid_phone,valid_address_line1,valid_area,valid_city,valid_country,in_stock,merchant_type,industry,ecommerce_type,weight,is_kit,sub_status
1537605,جنه زايد عماره ١٥ شقه ١,Apartment,6th of October City,0,Giza,2022-12-05 23:54:19.157565+00:00,Bahaa,0,False,349.0,soliman,Yaseen Mohieldeen(Gene),2022-12-08 14:20:34.154449+00:00,NaN,GNE-OT-2306,Bahaa soliman,2022-12-06,01005500224,NaN,Gene - Navy Brief 3-Pack- X-Large,All Navy Brief 3-Pack - XL,1,delivered,1536,Gene,2022-49,2022-49,31778986,8ac2c5e4-fa38-420d-b548-19a51a465e75,Gene,1.0,349.0,delivered,2022-12-05 23:54:19.157565+00:00,2022-12-06 16:34:05.436709+00:00,2022-12-08 09:42:45.854000+00:00,NaN,2022-12-05,2022-12-06,2022-12-08,NaN,Giza,1536.0,outbound_order,Bahaa soliman,01005500224,6th of October City,0.0,Bosta,31778986,Egypt,166.0,25822.0,cairo_giza,6th of October City,Giza,NaN,True,True,True,True,True,True,Brand,Fashion,shopify,2.0,NaN,NaN
1537606,بنها عزبة السلام - عمارة ١٤,NaN,بنها,0,قليوبية,2022-12-06 01:01:06.293657+00:00,Roaa,0,False,375.0,Ayman,Cleo Laboratories(cleo),2022-12-08 13:53:27.320235+00:00,NaN,CLO-OT-156888,Roaa Ayman,2022-12-06,01274810072,NaN,6224009604852,Hyaluronic Acid Sun Gel Cream SPF50+,1,delivered,1404,cleo,2022-49,2022-49,27005322,d8f19062-9a70-4fda-bebc-a5fd45debaf7,cleo,4.0,375.0,delivered,2022-12-06 01:01:06.293657+00:00,2022-12-07 17:03:02.756150+00:00,2022-12-08 12:17:45.763000+00:00,NaN,2022-12-06,2022-12-07,2022-12-08,NaN,قليوبية,1404.0,outbound_order,Roaa Ayman,01274810072,بنها,0.0,Bosta,27005322,EG,143.0,26126.0,delta_canal,Benha,El Kalioubia,NaN,True,True,True,True,True,True,Brand,Electronics,woo,2.0,NaN,NaN
1537607,بنها عزبة السلام - عمارة ١٤,NaN,بنها,0,قليوبية,2022-12-06 01:01:06.293657+00:00,Roaa,0,False,375.0,Ayman,Cleo Laboratories(cleo),2022-12-08 13:53:27.320235+00:00,NaN,CLO-OT-156888,Roaa Ayman,2022-12-06,01274810072,NaN,6224011102087,Lightening Eye Contour,1,delivered,1404,cleo,2022-49,2022-49,27005322,d8f19062-9a70-4fda-bebc-a5fd45debaf7,cleo,4.0,375.0,delivered,2022-12-06 01:01:06.293657+00:00,2022-12-07 17:03:02.756150+00:00,2022-12-08 12:17:45.763000+00:00,NaN,2022-12-06,2022-12-07,2022-12-08,NaN,قليوبية,1404.0,outbound_order,Roaa Ayman,01274810072,بنها,0.0,Bosta,27005322,EG,143.0,26126.0,delta_canal,Benha,El Kalioubia,NaN,True,True,True,True,True,True,Brand,Electronics,woo,2.0,NaN,NaN
1537608,بنها عزبة السلام - عمارة ١٤,NaN,بنها,0,قليوبية,2022-12-06 01:01:06.293657+00:00,Roaa,0,False,375.0,Ayman,Cleo Laboratories(cleo),2022-12-08 13:53:27.320235+00:00,NaN,CLO-OT-156888,Roaa Ayman,2022-12-06,01274810072,NaN,6224009604784,Anti-Aging Night Cream,1,delivered,1404,cleo,2022-49,2022-49,27005322,d8f19062-9a70-4fda-bebc-a5fd45debaf7,cleo,4.0,375.0,delivered,2022-12-06 01:01:06.293657+00:00,2022-12-07 17:03:02.756150+00:00,2022-12-08 12:17:45.763000+00:00,NaN,2022-12-06,2022-12-07,2022-12-08,NaN,قليوبية,1404.0,outbound_order,Roaa Ayman,01274810072,بنها,0.0,Bosta,27005322,EG,143.0,26126.0,delta_canal,Benha,El Kalioubia,NaN,True,True,True,True,True,True,Brand,Electronics,woo,2.0,NaN,NaN
1537609,بنها عزبة السلام - عمارة ١٤,NaN,بنها,0,قليوبية,2022-12-06 01:01:06.293657+00:00,Roaa,0,False,375.0,Ayman,Cleo Laboratories(cleo),2022-12-08 13:53:27.320235+00:00,NaN,CLO-OT-156888,Roaa Ayman,2022-12-06,01274810072,NaN,6224011102070,Vitamin C S

In [39]:
alkaseeba.head(1)

,order_code,Seller,Seller Mobile,Customer,Customer Mobile,Created At,SKU,Product Name,Quantity,Status,Price,Profit,Customer Address,In Stock,Customer Notes
0,129915,Mohamed essayed reda,1200010934,متولي عطا الله عبد المطلب,1001294859,2021-12-13 13:56:16.959453+00:00,MOB-NK,موبايل عفروتو - Mini Nokia 3310,1,delivered,390.0,135.0,عزبه حامد 2 بجوار خير زمان,out of stock,NaN


In [40]:
#new addition to change in kaseeba file for order_code strip string and active seller addition. New Kaseeba file does not have order code with BAS-OT string
df_merged['order_code_'] = df_merged['order_code'].str.rsplit('-', 1).str[-1]
alkaseeba['order_code_'] = alkaseeba['order_code'].str.rsplit('KSB', 1).str[-1]

#optional, if not joining on only numbers but on order_code column with BAS-OT appended back to 
# alkaseeba['order_code'] = alkaseeba['order_code'].str.rsplit('KSB', 1).str[-1]
alkaseeba['order_code'] = 'BAS-OT-' + alkaseeba['order_code'].astype(str)

In [41]:
alkaseeba.shape

(494461, 16)

Merge for either main order with Al-kaseeba or Al-kaseba with order

In [42]:
#compile the list of dataframes you want to merge first
#merge outbounds with main orders view table

# data_frames = [df_merged,main_orders]
# data_frames = [df_merged,alkaseeba]
# df_merged_2 = reduce(lambda left,right: pd.merge(left,right, on =['order_code'], how='left',suffixes = ("","_alkaseeba")), data_frames)

#if Kaseeba only
data_frames = [alkaseeba, df_merged]

# if merge on number only using order_code_ column
# df_merged_2 = reduce(lambda left,right: pd.merge(left,right, on =['order_code_'], how='left',suffixes = ("_alkaseeba","")), data_frames)

#if merge on BAS-OT- appended with number

#Merging outbouds with alkaseeba
df_merged_2 = pd.merge(df_merged, alkaseeba,  left_on='order_code',right_on='order_code', how = 'left',suffixes = ("","_alkaseeba"))


In [43]:
df_merged_2.shape

(1590130, 85)

In [44]:
df_merged_2.head(1)

,address_line1,address_line2,area,building_no,city,created_at,first_name,floor_no,is_work_address,items_cost,last_name,merchant_name,modified_at,note,order_code,order_customer_name,order_date,phone_number,secondary_phone_number,sku_code,sku_name,sku_pieces,status,user_id,username,week_no,week_number,tracking_number,order_id,merchant,total_quantity,items_cost_main,last_status,created_at_main,fulfilled_on,delivered_on,returned_to_origin_on,created_date,fulfilled_date,delivered_date,returned_to_origin_date,city_main,user_id_main,order_type,customer_name,phone_number_main,area_main,cash_returned,courier_name,tracking_number_main,country,location_id,invoice_id,zone_name,mapped_area,mapped_city,pickup_date,valid_phone,valid_address_line1,valid_area,valid_city,valid_country,in_stock,merchant_type,industry,ecommerce_type,weight,is_kit,sub_status,order_code_,Seller,Seller Mobile,Customer,Customer Mobile,Created At,SKU,Product Name,Quantity,Status,Price,Profit,Customer Address,In Stock,Customer Notes,order_code__alkaseeba
0,Address: الغربيه طنطا مركز قطور امام صدناوي شا...,NaN,Tanta,0,Gharbia,2021-06-20 14:32:54.086180+00:00,Salma Adel,NaN,False,465.0,.,Sara (rhea),2021-06-21 17:26:21.938044+00:00,NaN,RHE-OT-909991,Salma Adel .,2021-06-20,01022659816,NaN,K1,KERACELL SERUM,1,delivered,1057,rhea,2021-25,2021-25,46706446290,3d713ff6-9619-44ab-965e-eb5da73173a7,rhea,5.0,465.0,delivered,2021-06-20 14:32:54.086180+00:00,2021-06-21 11:10:16.151582+00:00,2021-06-22 10:32:00+00:00,NaN,2021-06-20,2021-06-21,2021-06-22,NaN,Gharbia,1057.0,outbound_order,Salma Adel .,01022659816,Tanta,0.0,Aramex,46706446290,Egypt,163.0,11772.0,delta_canal,Tanta,Gharbia,NaN,True,True,True,True,True,NaN,Brand,Cosmetics,NaN,1.6,NaN,NaN,909991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
# df_merged_2[df_merged_2.merchant_type == 'Affiliate']

In [46]:
#This represent that merging with kaseeba with outbounds orders will only will leave marketplace in merchant_type, therefore outbounds has been merged with Kaseeba

df_merged_2.merchant_type.unique()
df_merged_2.dropna(subset = 'merchant_type').merchant_type.unique()

array(['Brand', 'Seller', 'Marketplace', 'Affiliate'], dtype=object)

In [47]:
#Removing nans for merchant_type
df_merged_2.dropna(subset = 'merchant_type', inplace = True)
df_merged_2.merchant_type.unique()

array(['Brand', 'Seller', 'Marketplace', 'Affiliate'], dtype=object)

In [48]:
df_merged_2.shape

(1589583, 85)

In [49]:
df_merged_2.merchant_type.unique()

array(['Brand', 'Seller', 'Marketplace', 'Affiliate'], dtype=object)

In [50]:
# df_merged_2.head(1)
df_merged_2[df_merged_2.merchant_type == 'Brand']

,address_line1,address_line2,area,building_no,city,created_at,first_name,floor_no,is_work_address,items_cost,last_name,merchant_name,modified_at,note,order_code,order_customer_name,order_date,phone_number,secondary_phone_number,sku_code,sku_name,sku_pieces,status,user_id,username,week_no,week_number,tracking_number,order_id,merchant,total_quantity,items_cost_main,last_status,created_at_main,fulfilled_on,delivered_on,returned_to_origin_on,created_date,fulfilled_date,delivered_date,returned_to_origin_date,city_main,user_id_main,order_type,customer_name,phone_number_main,area_main,cash_returned,courier_name,tracking_number_main,country,location_id,invoice_id,zone_name,mapped_area,mapped_city,pickup_date,valid_phone,valid_address_line1,valid_area,valid_city,valid_country,in_stock,merchant_type,industry,ecommerce_type,weight,is_kit,sub_status,order_code_,Seller,Seller Mobile,Customer,Customer Mobile,Created At,SKU,Product Name,Quantity,Status,Price,Profit,Customer Address,In Stock,Customer Notes,order_code__alkaseeba
0,Address: الغربيه طنطا مركز قطور امام صدناوي شا...,NaN,Tanta,0,Gharbia,2021-06-20 14:32:54.086180+00:00,Salma Adel,NaN,False,465.0,.,Sara (rhea),2021-06-21 17:26:21.938044+00:00,NaN,RHE-OT-909991,Salma Adel .,2021-06-20,01022659816,NaN,K1,KERACELL SERUM,1,delivered,1057,rhea,2021-25,2021-25,46706446290,3d713ff6-9619-44ab-965e-eb5da73173a7,rhea,5.0,465.0,delivered,2021-06-20 14:32:54.086180+00:00,2021-06-21 11:10:16.151582+00:00,2021-06-22 10:32:00+00:00,NaN,2021-06-20,2021-06-21,2021-06-22,NaN,Gharbia,1057.0,outbound_order,Salma Adel .,01022659816,Tanta,0.0,Aramex,46706446290,Egypt,163.0,11772.0,delta_canal,Tanta,Gharbia,NaN,True,True,True,True,True,NaN,Brand,Cosmetics,NaN,1.60,NaN,NaN,909991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ابراج الري,بجوار قريه فينيسا,Shebeen Al-Kom,ابراج الري,Monufia,2021-07-11 10:50:39.672038+00:00,د.,NaN,False,230.0,معاذ مصطفي,Ahmed (elite),2021-12-08 10:19:32.347140+00:00,NaN,ELI-OT-EG10011939,د. معاذ مصطفي,2021-07-11,01020644556,NaN,139BK-2XL,shirt men desert black,1,returned to origin,1010,elite,2021-28,2021-28,46803081706,808d3b9c-66b3-4ec6-ad6c-124b214702e7,elite,1.0,230.0,returned to origin,2021-07-11 10:50:39.672038+00:00,2021-07-11 14:00:04.017300+00:00,NaN,2021-07-17 15:18:00+00:00,2021-07-11,2021-07-11,NaN,2021-07-17,Monufia,1010.0,outbound_order,د. معاذ مصطفي,01020644556,Shebeen Al-Kom,0.0,Aramex,46803081706,Egypt,257.0,11770.0,delta_canal,Shebeen Al-Kom,Monufia,NaN,True,True,True,True,True,NaN,Brand,Fashion,NaN,0.28,NaN,stocked,EG10011939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,الفيوم المحافظه نفسها عند صنيه دائري الحواتم,الفيوم المحافظه نفسها عند صنيه دائري الحواتم,الفيوم,1,الفيوم,2021-12-25 08:33:43.112929+00:00,احمد الشايب,1,False,299.0,احمد الشايب,Mohamed Marzok(Zohor),2021-12-25 09:31:49.952065+00:00,No Notes,ZHO-OT-15185,NaN,2021-12-24,01061595800,01061595800,406,عسل الطاقة,1,delivered,1269,Zohor,2021-51,2021-51,03QDf7fN,6f08a9f7-1c44-4528-82a5-7de920e75872,Zohor,1.0,299.0,delivered,2021-12-25 08:33:43.112929+00:00,2021-12-25 21:39:00.208146+00:00,2021-12-28 00:00:00+00:00,NaN,2021-12-25,2021-12-25,2021-12-28,NaN,الفيوم,1269.0,outbound_order,NaN,01061595800,الفيوم,0.0,R2S,03QDf7fN,Egypt,152.0,13541.0,upper_egypt,Fayoum,Fayoum,NaN,True,True,True,True,True,True,Brand,Trendy Products,NaN,1.00,NaN,NaN,15185,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,الحي السكني الاول المجاوره 6,مستشفى السلام الملكي,Qism Al-Raml 1,0,Alexandria,2022-05-04 15:45:36.993033+00:00,Manar,الارضي,False,155.0,Eisa,Mohamed Nasser(macro),2022-05-08 11:00:28.374679+00:00,العنوان مستشفى السلام الملكي,MCO-OT-21902,Manar Eisa,2022-05-04,01202621550,0112-444-4055,4251,Topi Gent Sebum Foam 150ml,2,delivered,1105,macro,2022-18,2022-18,80390877,b0b135bf-acf0-4565-bebf-b7f058b0b688,macro,2.0,155.0,delivered,2022-05-04 15:45:36.993033+00:00,2022-05-05 19:33:19.972483+00:00,2022-05-08 10:00:06.242000+00:00,NaN,2022-05-04,2022-05-05,2022-05-08,NaN,Alexa

In [51]:
#Removing Internal Sellers

In [52]:
#Condition to remove internal sellers from Kaseeba (for Egypt Only)
df_merged_2 = df_merged_2[~df_merged_2['Seller Mobile'].isin(kaseeba_internal_seller_egypt['Mobile Number'])]
df_merged_2.shape

(1569950, 85)

In [53]:
df_merged_2.merchant_type.unique()

array(['Brand', 'Seller', 'Marketplace', 'Affiliate'], dtype=object)

In [54]:
df_merged_2[df_merged_2.merchant_type == 'Seller']

,address_line1,address_line2,area,building_no,city,created_at,first_name,floor_no,is_work_address,items_cost,last_name,merchant_name,modified_at,note,order_code,order_customer_name,order_date,phone_number,secondary_phone_number,sku_code,sku_name,sku_pieces,status,user_id,username,week_no,week_number,tracking_number,order_id,merchant,total_quantity,items_cost_main,last_status,created_at_main,fulfilled_on,delivered_on,returned_to_origin_on,created_date,fulfilled_date,delivered_date,returned_to_origin_date,city_main,user_id_main,order_type,customer_name,phone_number_main,area_main,cash_returned,courier_name,tracking_number_main,country,location_id,invoice_id,zone_name,mapped_area,mapped_city,pickup_date,valid_phone,valid_address_line1,valid_area,valid_city,valid_country,in_stock,merchant_type,industry,ecommerce_type,weight,is_kit,sub_status,order_code_,Seller,Seller Mobile,Customer,Customer Mobile,Created At,SKU,Product Name,Quantity,Status,Price,Profit,Customer Address,In Stock,Customer Notes,order_code__alkaseeba
2,مدينة الرحاب ٢ مجموعة ١٣١ عمارة ١٩ الدور ٣ شقة ٣٤,NaN,New Cairo,19,Cairo,2021-10-16 10:13:07.452873+00:00,أحمد,3,False,1639.0,الخماش,Mahmoud (titansstore),2021-10-16 22:42:25.904305+00:00,NaN,TTN-OT-Order #2783,أحمد الخماش,2021-10-16,01122771132,NaN,HPX_ALLOYORIGINS60_RED,HyperX Alloy Origins 60% Mechanical Keyboard,1,delivered,1075,titansstore,2021-42,2021-42,1069522,b55c6130-bdb0-4043-acf6-4e9b8da6009b,titansstore,1.0,1639.0,delivered,2021-10-16 10:13:07.452873+00:00,2021-10-17 18:11:39.909939+00:00,2021-10-18 13:28:07+00:00,NaN,2021-10-16,2021-10-17,2021-10-18,NaN,Cairo,1075.0,outbound_order,أحمد الخماش,01122771132,New Cairo,0.0,Roadex,1069522,Egypt,105.0,11827.0,cairo_giza,New Cairo,Cairo,NaN,True,True,True,True,True,True,Seller,Electronics,NaN,NaN,NaN,NaN,Order #2783,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,العنوان بورسعيد- حي العرب- شارع رياض و طرح الب...,NaN,Qism Al-Arab,5,Port Said,2021-11-29 23:41:31.233199+00:00,Abdel Rahman,NaN,False,340.0,Elkhodary,Mohamed Abdelaziz(Styfol),2021-11-30 09:41:17.475932+00:00,NaN,STY-OT-492239,Abdel Rahman Elkhodary,2021-11-29,01090131413,NaN,Fred-M-W-1208,Fred Perry White - Medium,1,delivered,1120,Styfol,2021-48,2021-48,43872611042,0bb026d7-cb4c-4eb2-9681-951407286f63,Styfol,3.0,340.0,delivered,2021-11-29 23:41:31.233199+00:00,2021-11-30 18:46:56.832295+00:00,2021-12-01 09:48:27+00:00,NaN,2021-11-29,2021-11-30,2021-12-01,NaN,Port Said,1120.0,outbound_order,Abdel Rahman Elkhodary,01090131413,Qism Al-Arab,0.0,Aramex,43872611042,Egypt,272.0,12774.0,delta_canal,Qism Al-Arab,Port Said,NaN,True,True,True,True,True,True,Seller,Fashion,woo,1.14,NaN,NaN,492239,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,العنوان بورسعيد- حي العرب- شارع رياض و طرح الب...,NaN,Qism Al-Arab,5,Port Said,2021-11-29 23:41:31.233199+00:00,Abdel Rahman,NaN,False,340.0,Elkhodary,Mohamed Abdelaziz(Styfol),2021-11-30 09:41:17.475932+00:00,NaN,STY-OT-492239,Abdel Rahman Elkhodary,2021-11-29,01090131413,NaN,Fred-M-D-1204,Fred Perry Dark Blue - Medium,1,delivered,1120,Styfol,2021-48,2021-48,43872611042,0bb026d7-cb4c-4eb2-9681-951407286f63,Styfol,3.0,340.0,delivered,2021-11-29 23:41:31.233199+00:00,2021-11-30 18:46:56.832295+00:00,2021-12-01 09:48:27+00:00,NaN,2021-11-29,2021-11-30,2021-12-01,NaN,Port Said,1120.0,outbound_order,Abdel Rahman Elkhodary,01090131413,Qism Al-Arab,0.0,Aramex,43872611042,Egypt,272.0,12774.0,delta_canal,Qism Al-Arab,Port Said,NaN,True,True,True,True,True,True,Seller,Fashion,woo,1.14,NaN,NaN,492239,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,العنوان بورسعيد- حي العرب- شارع رياض و طرح الب...,NaN,Qism Al-Arab,5,Port Said,2021-11-29 23:41:31.233199+00:00,Abdel Rahman,NaN,False,340.0,Elkhodary,Mohamed Abdelaziz(Styfol),2021-11-30 09:41:17.475932+00:00,NaN,STY-OT-492239,Abdel Rahman Elkhodary,2021-11-29,01090131413,NaN,Fred-M-B-1200,Fred Perry Black - Medium,1,delivered,1120,Styfol,2021-48,2021-48,43872611042,0bb026d7-cb4c-4eb2-9681-9514072

In [55]:
df_merged_2.industry.unique()

array(['Cosmetics', 'Fashion', 'Electronics', 'Trendy Products', 'Books',
       'Marketplace', 'Home', 'Education', 'Entertainment', 'Misc',
       'Medical', 'Games', 'Sports Products', 'Heavy', 'Automotive',
       'Pet Care'], dtype=object)

In [56]:
df_merged_2.isna().sum()

address_line1                168
address_line2            1040826
area                        1866
building_no                17577
city                         495
                          ...   
Profit                   1321899
Customer Address         1321899
In Stock                 1321899
Customer Notes           1432565
order_code__alkaseeba    1321899
Length: 85, dtype: int64

In [57]:
# Remove Kaseeba or Saudi Kaseeba = #Affiliates
df_merged_2 = df_merged_2[df_merged_2.merchant_type != 'Marketplace']
df_merged_2 = df_merged_2[df_merged_2.merchant_type != 'Affiliate']

In [58]:
pd.set_option('display.max_rows',3000)
df_merged_2.isna().sum()

address_line1                  146
address_line2               582317
area                          1833
building_no                  15406
city                           445
created_at                       0
first_name                    2180
floor_no                    265623
is_work_address                  0
items_cost                       0
last_name                    62605
merchant_name                    0
modified_at                      0
note                        878759
order_code                       0
order_customer_name          29758
order_date                       0
phone_number                    40
secondary_phone_number      927931
sku_code                         0
sku_name                         0
sku_pieces                       0
status                           0
user_id                          0
username                         0
week_no                          0
week_number                      0
tracking_number              15859
order_id            

In [59]:
df_merged_2.shape

(1025227, 85)

In [60]:
df_merged_2.merchant_type.unique()

array(['Brand', 'Seller'], dtype=object)

In [61]:
df_merged_2.columns

Index(['address_line1', 'address_line2', 'area', 'building_no', 'city',
       'created_at', 'first_name', 'floor_no', 'is_work_address', 'items_cost',
       'last_name', 'merchant_name', 'modified_at', 'note', 'order_code',
       'order_customer_name', 'order_date', 'phone_number',
       'secondary_phone_number', 'sku_code', 'sku_name', 'sku_pieces',
       'status', 'user_id', 'username', 'week_no', 'week_number',
       'tracking_number', 'order_id', 'merchant', 'total_quantity',
       'items_cost_main', 'last_status', 'created_at_main', 'fulfilled_on',
       'delivered_on', 'returned_to_origin_on', 'created_date',
       'fulfilled_date', 'delivered_date', 'returned_to_origin_date',
       'city_main', 'user_id_main', 'order_type', 'customer_name',
       'phone_number_main', 'area_main', 'cash_returned', 'courier_name',
       'tracking_number_main', 'country', 'location_id', 'invoice_id',
       'zone_name', 'mapped_area', 'mapped_city', 'pickup_date', 'valid_phone',
       

In [62]:
df_merged_2.dropna(axis=1, how='all', inplace =True)
df_merged_2.columns

Index(['address_line1', 'address_line2', 'area', 'building_no', 'city',
       'created_at', 'first_name', 'floor_no', 'is_work_address', 'items_cost',
       'last_name', 'merchant_name', 'modified_at', 'note', 'order_code',
       'order_customer_name', 'order_date', 'phone_number',
       'secondary_phone_number', 'sku_code', 'sku_name', 'sku_pieces',
       'status', 'user_id', 'username', 'week_no', 'week_number',
       'tracking_number', 'order_id', 'merchant', 'total_quantity',
       'items_cost_main', 'last_status', 'created_at_main', 'fulfilled_on',
       'delivered_on', 'returned_to_origin_on', 'created_date',
       'fulfilled_date', 'delivered_date', 'returned_to_origin_date',
       'city_main', 'user_id_main', 'order_type', 'customer_name',
       'phone_number_main', 'area_main', 'cash_returned', 'courier_name',
       'tracking_number_main', 'country', 'location_id', 'invoice_id',
       'zone_name', 'mapped_area', 'mapped_city', 'pickup_date', 'valid_phone',
       

In [65]:
df_merged_2.isna().sum()

address_line1                  146
address_line2               582317
area                          1833
building_no                  15406
city                           445
created_at                       0
first_name                    2180
floor_no                    265623
is_work_address                  0
items_cost                       0
last_name                    62605
merchant_name                    0
modified_at                      0
note                        878759
order_code                       0
order_customer_name          29758
order_date                       0
phone_number                    40
secondary_phone_number      927931
sku_code                         0
sku_name                         0
sku_pieces                       0
status                           0
user_id                          0
username                         0
week_no                          0
week_number                      0
tracking_number              15859
order_id            

In [66]:
df_merged_2.merchant_type.unique()

array(['Brand', 'Seller'], dtype=object)

In [67]:
# df_merged_2[~df_merged_2['merchant_type'].str.contains('Marketplace') == True].sum()

In [68]:
outbounds.shape, main_orders.shape,  alkaseeba.shape, df_merged.shape, df_merged_2.shape

((1537610, 28), (1106385, 42), (494461, 16), (1537610, 70), (1025227, 70))

In [69]:
df_merged_2.merchant.unique()

array(['rhea', 'elite', 'titansstore', 'Styfol', 'Zohor', 'macro',
       'KING-WRAP', 'MoonLady', 'cleo', 'Blankiekids', 'Kijaqo',
       'Dermatique', 'ILady', 'Mindev', 'Pharmix', 'natavis', 'Scan.ksa',
       'RodyGallery', 'PhysicsSociety', 'RawAfrican', 'Pinkiego',
       'Yalladate', 'DrMohamedAyman', 'homepro', 'AbayaandGelbab',
       'ADshoecare', 'phoneixpharma', 'Tempo', 'Ziadex', 'LooliaCloset',
       'amitie', 'Minilet.co', 'bestseller', 'BodyBlaze', 'Cleanskin',
       'Dynasty', 'SohatiCare', 'Speaglish', 'donnanatural', 'dominohome',
       'keylash', 'Greatskirt', 'Shemsi', 'Bubblz', 'Jude', 'Zad',
       'Biobnz', 'Ones', 'Eyaclean', 'Eaglebrand', 'Goldenmark',
       'Gogreen', 'Riseoo', 'hibrew', 'Locken', 'Biostream',
       'Funnycouples', 'Englishmix', 'Nilco', 'bg-cosmetics', 'exercise',
       'earthowl', 'Beachill', 'ElMatgar', 'Dol', 'Scarpa', 'Gene',
       'EveSteve', 'Concrete', 'kashan', 'dynachem', 'qomaj', 'Ramo',
       'Nowet', 'mntjat26', 'beautyan

In [70]:
enas_sheet.shape

(852, 20)

In [71]:
enas_sheet.columns

Index(['SKU Name', 'Product Category', 'Subcategory', 'Season', 'Product Type',
       'Sells On', 'Kaseeba Code', 'Backyard Code', 'Min Profit', 'Min Price',
       'Backyard COD', 'Price 1', 'Price 2', 'Price 3', 'Avg Proc Price',
       'Last Proc Price', 'Sourcing Category', 'Lead Time Range',
       'Avg Lead Time', 'Vendor'],
      dtype='object')

In [72]:
#Adding product category and sub-category, however at the current time these are not maintained for merchant skus. 
# Done primarily to make sure, in case it is added in future it will automaticall be added.
# prod_cat_subcat = pd.read_csv("gs://flextock_ds_buuket/Demand Forecasting/TestData/Product_cat_subcat.csv", engine = "pyarrow")

In [73]:
prod_cat_subcat.shape

(1542, 20)

In [74]:
prod_cat_subcat.head(1)

,sku_name,sku_code,category,subcategory,is_active,pv_modified_at,ood_modified_at,variant_price,base_price,var_minimum_selling_price,suggested_price,product_id,id,product_name,subcategory_id,sub_cat_id,psc_category_id,category_id,date_diff,row_number
0,Variant Name,TEST-01,العناية بالسيارات,العناية بالسيارات,False,2023-01-09 14:46:44.117672+00:00,2021-12-01 13:53:12.184235+00:00,100.0,100.0,120.0,120.0,7,7,Test Product,110,110,42,42,"404 days, 0:53:31.933437",1


In [75]:
#merge enas sheet backyrard_code with merged df sku code
# df_merged_3 = pd.merge(df_merged_2,enas_sheet, left_on='sku_code',right_on='Backyard Code', how = 'left',suffixes = ("","_enas"))
df_merged_3 = pd.merge(df_merged_2,prod_cat_subcat, left_on='sku_code',right_on='sku_code', how = 'left',suffixes = ("","_category"))

In [76]:
df_merged_3.shape

(1025227, 89)

In [77]:
df_merged_3.head(2)

,address_line1,address_line2,area,building_no,city,created_at,first_name,floor_no,is_work_address,items_cost,last_name,merchant_name,modified_at,note,order_code,order_customer_name,order_date,phone_number,secondary_phone_number,sku_code,sku_name,sku_pieces,status,user_id,username,week_no,week_number,tracking_number,order_id,merchant,total_quantity,items_cost_main,last_status,created_at_main,fulfilled_on,delivered_on,returned_to_origin_on,created_date,fulfilled_date,delivered_date,returned_to_origin_date,city_main,user_id_main,order_type,customer_name,phone_number_main,area_main,cash_returned,courier_name,tracking_number_main,country,location_id,invoice_id,zone_name,mapped_area,mapped_city,pickup_date,valid_phone,valid_address_line1,valid_area,valid_city,valid_country,in_stock,merchant_type,industry,ecommerce_type,weight,is_kit,sub_status,order_code_,sku_name_category,category,subcategory,is_active,pv_modified_at,ood_modified_at,variant_price,base_price,var_minimum_selling_price,suggested_price,product_id,id,product_name,subcategory_id,sub_cat_id,psc_category_id,category_id,date_diff,row_number
0,Address: الغربيه طنطا مركز قطور امام صدناوي شا...,NaN,Tanta,0,Gharbia,2021-06-20 14:32:54.086180+00:00,Salma Adel,NaN,False,465.0,.,Sara (rhea),2021-06-21 17:26:21.938044+00:00,NaN,RHE-OT-909991,Salma Adel .,2021-06-20,01022659816,NaN,K1,KERACELL SERUM,1,delivered,1057,rhea,2021-25,2021-25,46706446290,3d713ff6-9619-44ab-965e-eb5da73173a7,rhea,5.0,465.0,delivered,2021-06-20 14:32:54.086180+00:00,2021-06-21 11:10:16.151582+00:00,2021-06-22 10:32:00+00:00,NaN,2021-06-20,2021-06-21,2021-06-22,NaN,Gharbia,1057.0,outbound_order,Salma Adel .,01022659816,Tanta,0.0,Aramex,46706446290,Egypt,163.0,11772.0,delta_canal,Tanta,Gharbia,NaN,True,True,True,True,True,NaN,Brand,Cosmetics,NaN,1.60,NaN,NaN,909991,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ابراج الري,بجوار قريه فينيسا,Shebeen Al-Kom,ابراج الري,Monufia,2021-07-11 10:50:39.672038+00:00,د.,NaN,False,230.0,معاذ مصطفي,Ahmed (elite),2021-12-08 10:19:32.347140+00:00,NaN,ELI-OT-EG10011939,د. معاذ مصطفي,2021-07-11,01020644556,NaN,139BK-2XL,shirt men desert black,1,returned to origin,1010,elite,2021-28,2021-28,46803081706,808d3b9c-66b3-4ec6-ad6c-124b214702e7,elite,1.0,230.0,returned to origin,2021-07-11 10:50:39.672038+00:00,2021-07-11 14:00:04.017300+00:00,NaN,2021-07-17 15:18:00+00:00,2021-07-11,2021-07-11,NaN,2021-07-17,Monufia,1010.0,outbound_order,د. معاذ مصطفي,01020644556,Shebeen Al-Kom,0.0,Aramex,46803081706,Egypt,257.0,11770.0,delta_canal,Shebeen Al-Kom,Monufia,NaN,True,True,True,True,True,NaN,Brand,Fashion,NaN,0.28,NaN,stocked,EG10011939,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [78]:
df_merged_3[df_merged_3.category.isnull() == False]

,address_line1,address_line2,area,building_no,city,created_at,first_name,floor_no,is_work_address,items_cost,last_name,merchant_name,modified_at,note,order_code,order_customer_name,order_date,phone_number,secondary_phone_number,sku_code,sku_name,sku_pieces,status,user_id,username,week_no,week_number,tracking_number,order_id,merchant,total_quantity,items_cost_main,last_status,created_at_main,fulfilled_on,delivered_on,returned_to_origin_on,created_date,fulfilled_date,delivered_date,returned_to_origin_date,city_main,user_id_main,order_type,customer_name,phone_number_main,area_main,cash_returned,courier_name,tracking_number_main,country,location_id,invoice_id,zone_name,mapped_area,mapped_city,pickup_date,valid_phone,valid_address_line1,valid_area,valid_city,valid_country,in_stock,merchant_type,industry,ecommerce_type,weight,is_kit,sub_status,order_code_,sku_name_category,category,subcategory,is_active,pv_modified_at,ood_modified_at,variant_price,base_price,var_minimum_selling_price,suggested_price,product_id,id,product_name,subcategory_id,sub_cat_id,psc_category_id,category_id,date_diff,row_number
232,النزهة الجديدة شارع الشيخ علي الخفيف متفرع من ...,NaN,النزهة الجديدة,0,Cairo,2023-02-03 19:45:29.746187+00:00,Omar,0,False,650.0,Ahmed,Yasmine Elkeir(Jude),2023-02-06 13:00:23.545114+00:00,NaN,JUD-OT-J030223-14,Omar Ahmed,2023-02-03,01021918736,NaN,M10,Army Green,1,delivered,1199,Jude,2023-05,2023-05,47795869376,c62fddc3-07b5-4992-b86c-85055b02be59,Jude,2.0,650.0,delivered,2023-02-03 19:45:29.746187+00:00,2023-02-05 17:33:14.416509+00:00,2023-02-06 12:30:00+00:00,NaN,2023-02-03,2023-02-05,2023-02-06,NaN,Cairo,1199.0,outbound_order,Omar Ahmed,01021918736,النزهة الجديدة,0.0,Aramex,47795869376,EG,105.0,NaN,cairo_giza,New Cairo,Cairo,NaN,True,True,True,True,True,True,Brand,Fashion,woo,NaN,NaN,NaN,14,.,إلكترونيات,سماعات الاذن,False,2023-01-16 16:52:48.257172+00:00,NaT,0.0,NaN,0.0,NaN,1386.0,1386.0,سماعة ديجيتال M10,113.0,113.0,54.0,54.0,,1.0
35916,فيلا ٣٣. الياسمين ٧ - التسعين الشمالي,NaN,new cairo,0,Cairo,2023-02-05 14:45:19.970901+00:00,Marwa,0,False,0.0,Elshukri,Yasmine Elkeir(Jude),2023-02-07 21:00:24.783864+00:00,NaN,JUD-OT-J050223-12,Marwa Elshukri,2023-02-05,01000080308,NaN,M10,Army Green,1,in transit,1199,Jude,2023-06,2023-06,47796083086,d5d5b488-8305-4fdc-bb32-be427b828833,Jude,2.0,0.0,in transit,2023-02-05 14:45:19.970901+00:00,2023-02-07 18:34:53.691309+00:00,NaN,NaN,2023-02-05,2023-02-07,NaN,NaN,Cairo,1199.0,outbound_order,Marwa Elshukri,01000080308,new cairo,0.0,Aramex,47796083086,EG,105.0,NaN,cairo_giza,New Cairo,Cairo,NaN,True,True,True,True,True,True,Brand,Fashion,woo,NaN,NaN,at courier hub,12,.,إلكترونيات,سماعات الاذن,False,2023-01-16 16:52:48.257172+00:00,NaT,0.0,NaN,0.0,NaN,1386.0,1386.0,سماعة ديجيتال M10,113.0,113.0,54.0,54.0,,1.0
45304,"Villa 119, banafseg1 , abdelhameed gouda el sa...",NaN,"1st new cairo, near broccar",0,Cairo,2023-02-03 07:45:18.492519+00:00,Farah,0,False,1060.0,Farag,Yasmine Elkeir(Jude),2023-02-06 19:00:51.816593+00:00,NaN,JUD-OT-J030223-3,Farah Farag,2023-02-03,01010425975,NaN,M10,Army Green,1,delivered,1199,Jude,2023-05,2023-05,47795833562,24189882-181a-4b8f-b90c-f515b276b4b2,Jude,3.0,1060.0,delivered,2023-02-03 07:45:18.492519+00:00,2023-02-04 16:51:08.644188+00:00,2023-02-06 17:58:00+00:00,NaN,2023-02-03,2023-02-04,2023-02-06,NaN,Cairo,1199.0,outbound_order,Farah Farag,01010425975,"1st new cairo, near broccar",0.0,Aramex,47795833562,EG,105.0,NaN,cairo_giza,New Cairo,Cairo,NaN,True,True,True,True,True,True,Brand,Fashion,woo,NaN,NaN,NaN,3,.,إلكترونيات,سماعات الاذن,False,2023-01-16 16:52:48.257172+00:00,NaT,0.0,NaN,0.0,NaN,1386.0,1386.0,سماعة ديجيتال M10,113.0,113.0,54.0,54.0,,1.0
113139,١٨ شارع مرعي متفرع من علي عبد العزيز,NaN,Maadi,0,Cairo,2023-02-05 17:45:19.488192+00:00,Karima,0,False,0.0,Selim,Yasmine Elkeir(Jude),2023-02-07 08:00:05.833572+00:00,NaN,JUD-OT-J050223-15,Karima Selim,2023-02-05,01014465539,NaN,M10,Army Green,1,delivered,1199,Jude,2023-06,2023-06,47796083156,8f0fdfcc-0399-40b1-9

In [79]:
df_merged_3.notna().sum()

address_line1                1025081
address_line2                 442910
area                         1023394
building_no                  1009821
city                         1024782
created_at                   1025227
first_name                   1023047
floor_no                      759604
is_work_address              1025227
items_cost                   1025227
last_name                     962622
merchant_name                1025227
modified_at                  1025227
note                          146468
order_code                   1025227
order_customer_name           995469
order_date                   1025227
phone_number                 1025187
secondary_phone_number         97296
sku_code                     1025227
sku_name                     1025227
sku_pieces                   1025227
status                       1025227
user_id                      1025227
username                     1025227
week_no                      1025227
week_number                  1025227
t

In [80]:
# df_merged_3.dropna(subset = 'category').merchant_type.unique()
# df_merged_3.merchant_type.unique()
# df_merged_3.dropna(subset = 'category').industry.unique() 
df_merged_3.industry.unique()
# df_merged_3.sku_code.nunique()

array(['Cosmetics', 'Fashion', 'Electronics', 'Trendy Products', 'Home',
       'Education', 'Entertainment', 'Books', 'Misc', 'Medical',
       'Marketplace', 'Games', 'Sports Products', 'Heavy', 'Automotive',
       'Pet Care'], dtype=object)

In [81]:
df_merged_3.sku_code.to_csv('test.csv')

In [82]:
df_merged_3.last_status.unique()

array(['delivered', 'returned to origin', 'pending', 'in transit',
       'processing', 'ready', 'pick up', 'on hold', 'fulfilled', 'lost'],
      dtype=object)

In [83]:
df_merged_3['sku_code'].isna().sum()

0

In [84]:
df_merged_3.columns

Index(['address_line1', 'address_line2', 'area', 'building_no', 'city',
       'created_at', 'first_name', 'floor_no', 'is_work_address', 'items_cost',
       'last_name', 'merchant_name', 'modified_at', 'note', 'order_code',
       'order_customer_name', 'order_date', 'phone_number',
       'secondary_phone_number', 'sku_code', 'sku_name', 'sku_pieces',
       'status', 'user_id', 'username', 'week_no', 'week_number',
       'tracking_number', 'order_id', 'merchant', 'total_quantity',
       'items_cost_main', 'last_status', 'created_at_main', 'fulfilled_on',
       'delivered_on', 'returned_to_origin_on', 'created_date',
       'fulfilled_date', 'delivered_date', 'returned_to_origin_date',
       'city_main', 'user_id_main', 'order_type', 'customer_name',
       'phone_number_main', 'area_main', 'cash_returned', 'courier_name',
       'tracking_number_main', 'country', 'location_id', 'invoice_id',
       'zone_name', 'mapped_area', 'mapped_city', 'pickup_date', 'valid_phone',
       

In [85]:
#below is identify what does merchant_type seller have?. To be looked into after asking from team
# df_merged_3[df_merged_3.merchant_type == 'Seller']

In [86]:
df_merged_3.industry.unique()
df_merged_3.weight.nunique()
df_merged_3.merchant_type.unique()
(df_merged_3.merchant_type == 'Seller').sum()

79951

In [87]:
len(df_merged_3)

1025227

In [88]:
df_merged_3.dropna(axis = 1, thresh =0.8*len(df_merged_3), inplace = True)

In [89]:
df_merged_3.head()

,address_line1,area,building_no,city,created_at,first_name,is_work_address,items_cost,last_name,merchant_name,modified_at,order_code,order_customer_name,order_date,phone_number,sku_code,sku_name,sku_pieces,status,user_id,username,week_no,week_number,tracking_number,order_id,merchant,total_quantity,items_cost_main,last_status,created_at_main,fulfilled_on,delivered_on,created_date,fulfilled_date,delivered_date,city_main,user_id_main,order_type,customer_name,phone_number_main,area_main,cash_returned,courier_name,tracking_number_main,country,location_id,invoice_id,zone_name,mapped_area,mapped_city,valid_phone,valid_address_line1,valid_area,valid_city,valid_country,in_stock,merchant_type,industry,order_code_
0,Address: الغربيه طنطا مركز قطور امام صدناوي شا...,Tanta,0,Gharbia,2021-06-20 14:32:54.086180+00:00,Salma Adel,False,465.0,.,Sara (rhea),2021-06-21 17:26:21.938044+00:00,RHE-OT-909991,Salma Adel .,2021-06-20,01022659816,K1,KERACELL SERUM,1,delivered,1057,rhea,2021-25,2021-25,46706446290,3d713ff6-9619-44ab-965e-eb5da73173a7,rhea,5.0,465.0,delivered,2021-06-20 14:32:54.086180+00:00,2021-06-21 11:10:16.151582+00:00,2021-06-22 10:32:00+00:00,2021-06-20,2021-06-21,2021-06-22,Gharbia,1057.0,outbound_order,Salma Adel .,01022659816,Tanta,0.0,Aramex,46706446290,Egypt,163.0,11772.0,delta_canal,Tanta,Gharbia,True,True,True,True,True,NaN,Brand,Cosmetics,909991
1,ابراج الري,Shebeen Al-Kom,ابراج الري,Monufia,2021-07-11 10:50:39.672038+00:00,د.,False,230.0,معاذ مصطفي,Ahmed (elite),2021-12-08 10:19:32.347140+00:00,ELI-OT-EG10011939,د. معاذ مصطفي,2021-07-11,01020644556,139BK-2XL,shirt men desert black,1,returned to origin,1010,elite,2021-28,2021-28,46803081706,808d3b9c-66b3-4ec6-ad6c-124b214702e7,elite,1.0,230.0,returned to origin,2021-07-11 10:50:39.672038+00:00,2021-07-11 14:00:04.017300+00:00,NaN,2021-07-11,2021-07-11,NaN,Monufia,1010.0,outbound_order,د. معاذ مصطفي,01020644556,Shebeen Al-Kom,0.0,Aramex,46803081706,Egypt,257.0,11770.0,delta_canal,Shebeen Al-Kom,Monufia,True,True,True,True,True,NaN,Brand,Fashion,EG10011939
2,مدينة الرحاب ٢ مجموعة ١٣١ عمارة ١٩ الدور ٣ شقة ٣٤,New Cairo,19,Cairo,2021-10-16 10:13:07.452873+00:00,أحمد,False,1639.0,الخماش,Mahmoud (titansstore),2021-10-16 22:42:25.904305+00:00,TTN-OT-Order #2783,أحمد الخماش,2021-10-16,01122771132,HPX_ALLOYORIGINS60_RED,HyperX Alloy Origins 60% Mechanical Keyboard,1,delivered,1075,titansstore,2021-42,2021-42,1069522,b55c6130-bdb0-4043-acf6-4e9b8da6009b,titansstore,1.0,1639.0,delivered,2021-10-16 10:13:07.452873+00:00,2021-10-17 18:11:39.909939+00:00,2021-10-18 13:28:07+00:00,2021-10-16,2021-10-17,2021-10-18,Cairo,1075.0,outbound_order,أحمد الخماش,01122771132,New Cairo,0.0,Roadex,1069522,Egypt,105.0,11827.0,cairo_giza,New Cairo,Cairo,True,True,True,True,True,True,Seller,Electronics,Order #2783
3,العنوان بورسعيد- حي العرب- شارع رياض و طرح الب...,Qism Al-Arab,5,Port Said,2021-11-29 23:41:31.233199+00:00,Abdel Rahman,False,340.0,Elkhodary,Mohamed Abdelaziz(Styfol),2021-11-30 09:41:17.475932+00:00,STY-OT-492239,Abdel Rahman Elkhodary,2021-11-29,01090131413,Fred-M-W-1208,Fred Perry White - Medium,1,delivered,1120,Styfol,2021-48,2021-48,43872611042,0bb026d7-cb4c-4eb2-9681-951407286f63,Styfol,3.0,340.0,delivered,2021-11-29 23:41:31.233199+00:00,2021-11-30 18:46:56.832295+00:00,2021-12-01 09:48:27+00:00,2021-11-29,2021-11-30,2021-12-01,Port Said,1120.0,outbound_order,Abdel Rahman Elkhodary,01090131413,Qism Al-Arab,0.0,Aramex,43872611042,Egypt,272.0,12774.0,delta_canal,Qism Al-Arab,Port Said,True,True,True,True,True,True,Seller,Fashion,492239
4,العنوان بورسعيد- حي العرب- شارع رياض و طرح الب...,Qism Al-Arab,5,Port Said,2021-11-29 23:41:31.233199+00:00,Abdel Rahman,False,340.0,Elkhodary,Mohamed Abdelaziz(Styfol),2021-11-30 09:41:17.475932+00:00,STY-OT-492239,Abdel Rahman Elkhodary,2021-11-29,01090131413,Fred-M-D-1204,Fred Perry Dark Blue - Medium,1,delivered,1120,Styfol,2021-48,2021-48,43872611042,0bb026d7-cb4c-4eb2-9681-951407286f63,Styfol,3.0,340.0,delivered,2021-11-29 23:41:31.233199+00:00,20

In [90]:
df_merged_3.columns

Index(['address_line1', 'area', 'building_no', 'city', 'created_at',
       'first_name', 'is_work_address', 'items_cost', 'last_name',
       'merchant_name', 'modified_at', 'order_code', 'order_customer_name',
       'order_date', 'phone_number', 'sku_code', 'sku_name', 'sku_pieces',
       'status', 'user_id', 'username', 'week_no', 'week_number',
       'tracking_number', 'order_id', 'merchant', 'total_quantity',
       'items_cost_main', 'last_status', 'created_at_main', 'fulfilled_on',
       'delivered_on', 'created_date', 'fulfilled_date', 'delivered_date',
       'city_main', 'user_id_main', 'order_type', 'customer_name',
       'phone_number_main', 'area_main', 'cash_returned', 'courier_name',
       'tracking_number_main', 'country', 'location_id', 'invoice_id',
       'zone_name', 'mapped_area', 'mapped_city', 'valid_phone',
       'valid_address_line1', 'valid_area', 'valid_city', 'valid_country',
       'in_stock', 'merchant_type', 'industry', 'order_code_'],
      dtype=

In [91]:
outbounds.created_at.min(), main_orders.created_at.min(),df_merged.created_at.min(), df_merged.order_date.min()

# df_merged_3['order_date'] = pd.to_datetime(df_merged_3['order_date'],errors = 'coerce')
# df_merged_3 = df_merged_3[df_merged_3.order_date.dt.date.between(date.today() - pd.DateOffset(730), date.today())]

#convert created_at column to type datetime
df_merged_3['created_at'] = pd.to_datetime(df_merged_3['created_at'],errors = 'coerce')
#for removing erroneous dates
df_merged_3 = df_merged_3[df_merged_3.created_at.dt.date.between(date.today() - pd.DateOffset(730), date.today())]

In [92]:
df_merged_3.created_at = pd.to_datetime(df_merged_3.created_at.dt.date)

In [93]:
#testing
dates_0.dtypes
df_merged_3.dtypes
# dates_0.Join_Dates.min()
# df_merged_3.order_date.min()
# df_merged_3.order_date.sort_values()
# df_merged_3.created_at.sort_values()

address_line1                   object
area                            object
building_no                     object
city                            object
created_at              datetime64[ns]
first_name                      object
is_work_address                   bool
items_cost                     float64
last_name                       object
merchant_name                   object
modified_at                     object
order_code                      object
order_customer_name             object
order_date                      object
phone_number                    object
sku_code                        object
sku_name                        object
sku_pieces                       int64
status                          object
user_id                          int64
username                        object
week_no                         object
week_number                     object
tracking_number                 object
order_id                        object
merchant                 

In [94]:
#merge to get uniform dates
# df_merged_3 = pd.merge(dates_0,df_merged_3,left_on=['Join_Dates'],right_on=['order_date'], how = 'left')
df_merged_3 = pd.merge(dates_0,df_merged_3,left_on=['Join_Dates'],right_on=['created_at'], how = 'left')
#rename column to Uniform_Date
df_merged_3.rename(columns = {'Join_Dates':'Uniform_Date'}, inplace = True)

In [95]:
#testing
# dates_0.dtypes
# df_merged_3.dtypes
# dates_0.Join_Dates.min()
# df_merged_3.order_date.min()
# # df_merged_3.order_date.sort_values()
df_merged_3.columns

Index(['Uniform_Date', 'address_line1', 'area', 'building_no', 'city',
       'created_at', 'first_name', 'is_work_address', 'items_cost',
       'last_name', 'merchant_name', 'modified_at', 'order_code',
       'order_customer_name', 'order_date', 'phone_number', 'sku_code',
       'sku_name', 'sku_pieces', 'status', 'user_id', 'username', 'week_no',
       'week_number', 'tracking_number', 'order_id', 'merchant',
       'total_quantity', 'items_cost_main', 'last_status', 'created_at_main',
       'fulfilled_on', 'delivered_on', 'created_date', 'fulfilled_date',
       'delivered_date', 'city_main', 'user_id_main', 'order_type',
       'customer_name', 'phone_number_main', 'area_main', 'cash_returned',
       'courier_name', 'tracking_number_main', 'country', 'location_id',
       'invoice_id', 'zone_name', 'mapped_area', 'mapped_city', 'valid_phone',
       'valid_address_line1', 'valid_area', 'valid_city', 'valid_country',
       'in_stock', 'merchant_type', 'industry', 'order_code_

In [96]:
#re-arrange columns
df_merged_3.columns

cols_arranged = ['Uniform_Date','order_type','order_id','order_code','order_code_','sku_code','sku_name','merchant','merchant_name', 'merchant_type', 'industry','sku_pieces',
        'items_cost','items_cost_main','cash_returned',
        'status','last_status',
        'in_stock', 
        'order_date','created_at','created_at_main', 'created_date' ,'fulfilled_on', 'fulfilled_date', 'modified_at', 'delivered_date',
        'area', 'area_main', 'mapped_area', 'valid_area','building_no', 'city', 'city_main', 'mapped_city','zone_name', 'valid_city', 'valid_country', 'country', 'location_id', 'invoice_id', 'valid_phone', 'address_line1','valid_address_line1','is_work_address', 
        'user_id_main',  'customer_name', 'phone_number_main',
        'courier_name', 'tracking_number', 'tracking_number_main',
        'week_no', 'week_number']

cols_cleaned = ['Uniform_Date','order_type','order_id','order_code','sku_code','sku_name','merchant','merchant_name', 'merchant_type', 'industry','sku_pieces',
        'items_cost','cash_returned',
        'last_status',
        'in_stock', 
        'order_date','created_at', 'created_date' ,'fulfilled_on', 'fulfilled_date', 'modified_at', 'delivered_date',
        'area', 'mapped_area', 'valid_area','building_no', 'city', 'mapped_city','zone_name', 'valid_city', 'valid_country', 'country', 'location_id', 'invoice_id', 'valid_phone', 'address_line1','valid_address_line1','is_work_address', 
        'user_id_main',  'customer_name', 'phone_number_main',
        'courier_name', 'tracking_number',
        'week_number']

df_merged_3 = df_merged_3[cols_cleaned]
df_merged_3

,Uniform_Date,order_type,order_id,order_code,sku_code,sku_name,merchant,merchant_name,merchant_type,industry,sku_pieces,items_cost,cash_returned,last_status,in_stock,order_date,created_at,created_date,fulfilled_on,fulfilled_date,modified_at,delivered_date,area,mapped_area,valid_area,building_no,city,mapped_city,zone_name,valid_city,valid_country,country,location_id,invoice_id,valid_phone,address_line1,valid_address_line1,is_work_address,user_id_main,customer_name,phone_number_main,courier_name,tracking_number,week_number
0,2021-02-08,outbound_order,fad5f9a2-30a2-4eda-980d-8dc6de6fe3e4,FIF-OT-FIF-1263,515KBA27,Basic Unisex Charcoal Hoodie - XL,fif,Fouad (fif),Brand,Fashion,1.0,515.0,0.0,delivered,NaN,2021-02-08,2021-02-08,2021-02-08,2021-02-09 00:00:00+00:00,2021-02-09,2021-02-08 11:42:11.249284+00:00,2021-02-10,Zamalek,Zamalek,True,4,Cairo,Cairo,cairo_giza,True,True,Egypt,113.0,11843.0,True,4 omaret al yamani Street Ground floor,True,False,1001.0,Ahmed Talaat,01001611604,Loadbugs,140981,2021-06
1,2021-02-08,outbound_order,63496d77-86d1-4345-8a46-4400a0f21a38,FIF-OT-FIF-2000,FIHBXSW,Beirut Hoodie-XS-White,fif,Fouad (fif),Brand,Fashion,1.0,0.0,0.0,delivered,NaN,2021-02-08,2021-02-08,2021-02-08,2021-02-09 00:00:00+00:00,2021-02-09,2021-02-08 11:50:42.125285+00:00,2021-02-10,Nasr City,Nasr City,True,24,Cairo,Cairo,cairo_giza,True,True,Egypt,104.0,11843.0,True,24 Ahmed Fakhry Street,True,False,1001.0,Sarah Mohamed,01286590341,Loadbugs,140982,2021-06
2,2021-02-08,outbound_order,63496d77-86d1-4345-8a46-4400a0f21a38,FIF-OT-FIF-2000,FISLWLG,Lil Wayne Sweater-L-Grey,fif,Fouad (fif),Brand,Fashion,1.0,0.0,0.0,delivered,NaN,2021-02-08,2021-02-08,2021-02-08,2021-02-09 00:00:00+00:00,2021-02-09,2021-02-08 11:50:42.125285+00:00,2021-02-10,Nasr City,Nasr City,True,24,Cairo,Cairo,cairo_giza,True,True,Egypt,104.0,11843.0,True,24 Ahmed Fakhry Street,True,False,1001.0,Sarah Mohamed,01286590341,Loadbugs,140982,2021-06
3,2021-02-08,outbound_order,b53b99fe-a36e-47be-a5b9-ed3e91109e2a,HIB-OT-401628,CMH001,HiBrew Coffee Machine,hibrew,HiBREW (hibrew),Brand,Home,1.0,0.0,0.0,pick up,NaN,2021-02-08,2021-02-08,2021-02-08,2021-02-08 00:00:00+00:00,2021-02-08,2021-07-07 16:17:43.511658+00:00,NaN,3rd Settlement,NaN,False,Warehouse Pickup,Cairo,Cairo,cairo_giza,True,True,Egypt,NaN,11863.0,True,Warehouse Pickup,False,False,1004.0,David Azmy,01234567890,Flextock (Same day),Pick up,2021-06
4,2021-02-08,outbound_order,05e3f750-4fa1-4c9e-8b78-1f15d630af0e,FIF-OT-FIF-1252_1,3H6F4MB2,FIF Basic Unisex Black On Black Sweatpants Gre...,fif,Fouad (fif),Brand,Fashion,1.0,0.0,0.0,delivered,NaN,2021-02-08,2021-02-08,2021-02-08,2021-02-10 00:00:00+00:00,2021-02-10,2021-02-08 13:24:56.982701+00:00,2021-02-11,Heliopolis,Heliopolis,True,97,Cairo,Cairo,cairo_giza,True,True,Egypt,99.0,11843.0,True,97 elmarghany street masr el gdeda,True,False,1001.0,Omar Mahsoub,01005332092,Loadbugs,141086,2021-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1024896,2023-02-07,outbound_order,f3824c48-7e1c-4b49-95dd-7429bdfc66c0,BND-OT-6280,10131038,Be-Strong Blue Tie-dye - 38,Beindie,Farah (Beindie),Brand,Fashion,1.0,1300.0,0.0,pending,True,2023-02-07,2023-02-07,2023-02-07,NaN,NaN,2023-02-07 22:27:36.048062+00:00,NaN,New Cairo,New Cairo,True,0,Cairo,Cairo,cairo_giza,True,True,Egypt,105.0,NaN,True,Compound Lakeview,True,False,1628.0,Emmy Tarek,01018848473,NaN,NaN,2023-06
1024897,2023-02-07,outbound_order,205151a1-7cb6-4bc6-b0b1-cdb188240501,ZHO-OT-66148,101,السبيرولينا,Zohor,Mohamed Marzok(Zohor),Brand,Trendy Products,1.0,345.0,0.0,processing,True,2023-02-07,2023-02-07,2023-02-07,NaN,NaN,2023-02-07 10:10:35.900169+00:00,NaN,الجيزة,Agouza,True,1,الهرم,Giza,cairo_giza,True,True,Egypt,168.0,NaN,True,محافظة الجيزة - الهرم - ٨٣ شارع الهرم - فوق نف...,True,False,1269.0,NaN,01007912692,JT Express,JEG000033457762,2023-06
1024898,2023-02-07,outbound_order,63e0eb01-105e-4fb4

In [97]:
# from Pyc_functions import get_parsimonious_data
# get_parsimonious_data(0.9, df_merged_3)


### Adding sku wise DR

In [98]:
#changed from 'status_unique to 'last_status_unique'
sku_dr = df_merged_3[['sku_code','created_at','week_number','last_status']]

In [99]:
sku_dr

,sku_code,created_at,week_number,last_status
0,515KBA27,2021-02-08,2021-06,delivered
1,FIHBXSW,2021-02-08,2021-06,delivered
2,FISLWLG,2021-02-08,2021-06,delivered
3,CMH001,2021-02-08,2021-06,pick up
4,3H6F4MB2,2021-02-08,2021-06,delivered
...,...,...,...,...
1024896,10131038,2023-02-07,2023-06,pending
1024897,101,2023-02-07,2023-06,processing
1024898,em-0088,2023-02-07,2023-06,ready
1024899,MB049201,2023-02-07,2023-06,processing


In [100]:
sku_dr.dropna(inplace = True)
sku_dr.created_at = pd.to_datetime(sku_dr.created_at).dt.date
sku_dr.head()

,sku_code,created_at,week_number,last_status
0,515KBA27,2021-02-08,2021-06,delivered
1,FIHBXSW,2021-02-08,2021-06,delivered
2,FISLWLG,2021-02-08,2021-06,delivered
3,CMH001,2021-02-08,2021-06,pick up
4,3H6F4MB2,2021-02-08,2021-06,delivered


In [101]:
sku_dr = pd.crosstab([sku_dr['sku_code'],sku_dr['created_at']],sku_dr['last_status'],values = sku_dr['last_status'], aggfunc='count',normalize='index')
sku_dr

last_status           delivered  fulfilled  in transit  lost  on hold  \
sku_code  created_at                                                    
 EH-30059 2021-11-01        1.0        0.0         0.0   0.0      0.0   
          2022-04-17        1.0        0.0         0.0   0.0      0.0   
#001BL    2022-05-09        1.0        0.0         0.0   0.0      0.0   
          2022-05-23        1.0        0.0         0.0   0.0      0.0   
          2022-05-28        1.0        0.0         0.0   0.0      0.0   
...                         ...        ...         ...   ...      ...   
٤٤٤       2021-08-05        1.0        0.0         0.0   0.0      0.0   
٥         2021-07-27        0.0        0.0         0.0   0.0      0.0   
٦         2021-07-27        0.0        0.0         0.0   0.0      0.0   
٧         2021-07-27        0.0        0.0         0.0   0.0      0.0   
◼️ الدكان 2021-08-21        1.0        0.0         0.0   0.0      0.0   

last_status           pending  pick up  processing  ready  returned to origin  
sku_code  created_at                                                           
 EH-30059 2021-11-01      0.0      0.0         0.0    0.0                 0.0  
          2022-04-17      0.0      0.0         0.0    0.0                 0.0  
#001BL    2022-05-09      0.0      0.0         0.0    0.0                 0.0  
          2022-05-23      0.0      0.0         0.0    0.0                 0.0  
          2022-05-28      0.0      0.0         0.0    0.0                 0.0  
...                       ...      ...         ...    ...                 ...  
٤٤٤       2021-08-05      0.0      0.0         0.0    0.0                 0.0  
٥         2021-07-27      0.0      0.0         0.0    0.0                 1.0  
٦         2021-07-27      0.0      0.0         0.0    0.0                 1.0  
٧         2021-07-27      0.0      0.0         0.0    0.0                 1.0  
◼️ الدكان 2021-08-21      0.0      0.0         0.0    0.0                 0.0  

[254129 rows x 10 columns]

In [102]:
sku_dr.reset_index(inplace = True)
sku_dr

last_status,sku_code,created_at,delivered,fulfilled,in transit,lost,on hold,pending,pick up,processing,ready,returned to origin
0,EH-30059,2021-11-01,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,EH-30059,2022-04-17,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,#001BL,2022-05-09,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,#001BL,2022-05-23,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,#001BL,2022-05-28,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
254124,٤٤٤,2021-08-05,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
254125,٥,2021-07-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
254126,٦,2021-07-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
254127,٧,2021-07-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [103]:
sku_dr['created_at'] = pd.to_datetime(sku_dr['created_at'])
sku_dr.dtypes

last_status
sku_code                      object
created_at            datetime64[ns]
delivered                    float64
fulfilled                    float64
in transit                   float64
lost                         float64
on hold                      float64
pending                      float64
pick up                      float64
processing                   float64
ready                        float64
returned to origin           float64
dtype: object

In [104]:
sku_dr = pd.merge(dates_0,sku_dr,left_on=['Join_Dates'],right_on=['created_at'], how = 'left')
sku_dr

,Join_Dates,sku_code,created_at,delivered,fulfilled,in transit,lost,on hold,pending,pick up,processing,ready,returned to origin
0,2021-02-08,31RL9YTE,2021-02-08,1.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0
1,2021-02-08,3H6F4MB2,2021-02-08,1.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0
2,2021-02-08,515KBA27,2021-02-08,1.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0
3,2021-02-08,CMH001,2021-02-08,0.0,0.0,0.00,0.0,0.0,0.0,1.0,0.00,0.0,0.0
4,2021-02-08,FIHBXSW,2021-02-08,1.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
254125,2023-02-07,موديل نيللي ابيض XXXL,2023-02-07,0.0,0.0,1.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0
254126,2023-02-07,موديل نيللي اسود L,2023-02-07,0.0,0.0,0.75,0.0,0.0,0.0,0.0,0.25,0.0,0.0
254127,2023-02-07,موديل نيللي اسود XXL,2023-02-07,0.0,0.0,0.75,0.0,0.0,0.0,0.0,0.25,0.0,0.0
254128,2023-02-07,موديل نيللي جنزاري XXL,2023-02-07,0.0,0.0,0.50,0.0,0.0,0.0,0.0,0.50,0.0,0.0


In [105]:
sku_dr.dropna(subset = ['sku_code'], inplace = True)

In [106]:
sku_dr

,Join_Dates,sku_code,created_at,delivered,fulfilled,in transit,lost,on hold,pending,pick up,processing,ready,returned to origin
0,2021-02-08,31RL9YTE,2021-02-08,1.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0
1,2021-02-08,3H6F4MB2,2021-02-08,1.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0
2,2021-02-08,515KBA27,2021-02-08,1.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0
3,2021-02-08,CMH001,2021-02-08,0.0,0.0,0.00,0.0,0.0,0.0,1.0,0.00,0.0,0.0
4,2021-02-08,FIHBXSW,2021-02-08,1.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
254125,2023-02-07,موديل نيللي ابيض XXXL,2023-02-07,0.0,0.0,1.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0
254126,2023-02-07,موديل نيللي اسود L,2023-02-07,0.0,0.0,0.75,0.0,0.0,0.0,0.0,0.25,0.0,0.0
254127,2023-02-07,موديل نيللي اسود XXL,2023-02-07,0.0,0.0,0.75,0.0,0.0,0.0,0.0,0.25,0.0,0.0
254128,2023-02-07,موديل نيللي جنزاري XXL,2023-02-07,0.0,0.0,0.50,0.0,0.0,0.0,0.0,0.50,0.0,0.0


##################################################################################################################################################################################################################################
##################################################################################################################################################################################################################################

## Inventory (Mapping + Dynamic SKUs)

In [107]:
#iventory data for sku code and quantity
inventory.head()

,merchant,SKU Name,SKU Code,All Barcodes,Location,qty
0,21stiches,Donda hoodie M,21S-21dondaM,['35571846'],Z5-A32-L3-B06,5
1,21stiches,Donda hoodie M,21S-21dondaM,['35571846'],Z6-A22-L2-B08,1
2,21stiches,Don Hoodie L,21S-21donL,['96585662'],Z6-A21-L5-B07,1
3,21stiches,Don hoodie M,21S-21donM,['98244989'],Z5-A32-L4-B06,3
4,21stiches,Nike jacket L,21S-21nikejackL,['68981658'],Z5-A22-L3-B11,1


In [108]:
#unique count of merchants in inventory
inventory.merchant.nunique()

239

In [109]:
#unique count of merchants in df_merged_3
df_merged_3.merchant.nunique()

302

In [110]:
#unique count of common merchants in inventory & df_merged_3
len(set(inventory.merchant) & set(df_merged_3.merchant))

213

In [111]:
#unique count of common merchants in inventory & df_merged_3
len(set(inventory.merchant) & set(df_merged_3.merchant))

213

In [112]:
inventory.merchant.unique(), df_merged_3.merchant.unique()

(array(['21stiches', '2oolameme', '3almoda', '80Fekra', 'ArabicGum',
        'AbayaandGelbab', 'Acacia', 'AfricanGreen', 'Ageba',
        'AngelsBeauty', 'amitie', 'April', 'areaunique', 'ares', 'Arihalk',
        'ADshoecare', 'atlassources', 'azShop', 'BackyardSale-SK',
        'Bluebells', 'BabyGarage', 'Bubblz', 'TheBasicLook', 'BodyBlaze',
        'Beachill', 'beenaorganics', 'BeautySook', 'BeeEgypt',
        'bg-cosmetics', 'Biobnz', 'Backyardsale-Saudi', 'Blankiekids',
        'BellaMall', 'BMarket', 'beautyandbeyond', 'Beindie', 'Beautinova',
        'Biostream', 'beitekstore', 'TheBathland', 'BeeWaxEgypt',
        'BOOXTORE', 'Booxtore', 'Bayshoes', 'ChampionsStore', 'cleo',
        'Cleanskin', 'Cento.outfit', 'Concrete', 'Carbonpress', 'Cosmella',
        'cotojama', 'DokanMarwa', 'DrMohamedAyman', 'donnanatural', 'Dol',
        'Dott', 'Dripscorner', 'darlida', 'Dermatique', 'dynachem',
        'Dynasty', 'egyptbobbers', 'Eleganta', 'Eaglebrand', 'KING-WRAP',
        'Elle'

In [113]:
# inventory_mapping=pd.read_csv("gs://flextock_ds_buuket/Demand Forecasting/TestData/inventory_mapping.csv", engine = "pyarrow")

# dynamic_skus = pd.read_csv("gs://flextock_ds_buuket/Demand Forecasting/TestData/dynamic_inventory_all.csv", engine = "pyarrow")

In [114]:
inventory_mapping.head(1)

,sku_code,wms_sku_code,name,is_active
0,4000000086536,con-4000000086536,C112TR2-W22-70W-1012,True


In [115]:
dynamic_skus.head(1)

,sku_code,transaction_date,sum
0,21S-21dondaL,2021-12-06,7


In [116]:
#Dynamic iventory data for sku code and quantity
dynamic_skus.rename(columns = {'sum':'QTY', 'sku_code':'SKU Code'}, inplace = True)
dynamic_skus.head(1)

,SKU Code,transaction_date,QTY
0,21S-21dondaL,2021-12-06,7


In [117]:
inventory_mapping.tail(1)

,sku_code,wms_sku_code,name,is_active
87068,mak1,YRQA-mak1,منظف الكترونيات,True


In [118]:
#Inventory Mapping directly from source needs to have split for wms_sku_code also since it has '-' in it to 
#Only apply if inventory mapping file is directly being sourced
inventory_mapping[['wms_sku_code_p1','wms_sku_code_p2']] = inventory_mapping['wms_sku_code'].str.split('-', 1, expand=True)


In [119]:
inventory_mapping.tail()

,sku_code,wms_sku_code,name,is_active,wms_sku_code_p1,wms_sku_code_p2
87064,6224009604869,DRM-6224009604869,Micellar Water,True,DRM,6224009604869
87065,104,YRQA-104,حامل هاتف ذكي-احمر,True,YRQA,104
87066,109,YRQA-109,تعليقة البطة (Gypsum Swing)-نظارة,True,YRQA,109
87067,mgb,YRQA-mgb,معطر هواء من baseus,True,YRQA,mgb
87068,mak1,YRQA-mak1,منظف الكترونيات,True,YRQA,mak1


In [120]:
# inventory[['SKU_Code_p1','SKU_Code_p2']] = inventory['SKU Code'].str.split('-', 1, expand=True)
dynamic_skus[['SKU_Code_p1','SKU_Code_p2']] = dynamic_skus['SKU Code'].str.split('-', 1, expand=True)

In [121]:
# inventory.head()

In [122]:
dynamic_skus.head()

,SKU Code,transaction_date,QTY,SKU_Code_p1,SKU_Code_p2
0,21S-21dondaL,2021-12-06,7,21S,21dondaL
1,21S-21dondaL,2021-12-09,5,21S,21dondaL
2,21S-21dondaL,2021-12-11,4,21S,21dondaL
3,21S-21dondaL,2021-12-16,3,21S,21dondaL
4,21S-21dondaL,2021-12-22,0,21S,21dondaL


In [123]:
#link inventory and inventory mapping sheet
# df_merged_4 = pd.merge(inventory,inventory_mapping, left_on='SKU_Code_p2' ,right_on='wms_sku_code', how = 'left',suffixes = ("","_mapping"))
df_merged_4 = pd.merge(dynamic_skus,inventory_mapping, left_on='SKU_Code_p2' ,right_on='wms_sku_code_p2', how = 'left',suffixes = ("","_mapping"))

In [124]:
df_merged_4.head(5)

,SKU Code,transaction_date,QTY,SKU_Code_p1,SKU_Code_p2,sku_code,wms_sku_code,name,is_active,wms_sku_code_p1,wms_sku_code_p2
0,21S-21dondaL,2021-12-06,7,21S,21dondaL,21dondaL,21S-21dondaL,Donda hoodie L,True,21S,21dondaL
1,21S-21dondaL,2021-12-09,5,21S,21dondaL,21dondaL,21S-21dondaL,Donda hoodie L,True,21S,21dondaL
2,21S-21dondaL,2021-12-11,4,21S,21dondaL,21dondaL,21S-21dondaL,Donda hoodie L,True,21S,21dondaL
3,21S-21dondaL,2021-12-16,3,21S,21dondaL,21dondaL,21S-21dondaL,Donda hoodie L,True,21S,21dondaL
4,21S-21dondaL,2021-12-22,0,21S,21dondaL,21dondaL,21S-21dondaL,Donda hoodie L,True,21S,21dondaL


In [125]:
df_merged_4['SKU_Code_p2'].nunique()

35353

In [126]:
df_merged_4['sku_code'].nunique()

35661

In [127]:
#use SKU CODE and where SKU CODE is not available retain SKU_Code_p2
#comment out if old logic is to be replaced and new file has come in

df_merged_4['SKU_Code_p2'] = (df_merged_4['sku_code'].fillna(df_merged_4['SKU_Code_p2']))

In [128]:
df_merged_4['SKU_Code_p2'].nunique()

35674

In [129]:
df_merged_5 = df_merged_4.groupby(['SKU_Code_p2','transaction_date'])['QTY'].sum().reset_index()

In [130]:
df_merged_5.head(2)

,SKU_Code_p2,transaction_date,QTY
0,EH-30059,2021-11-02,0
1,EH-30059,2022-04-18,1


In [131]:
df_merged_5['SKU_Code_p2'].shape

(528839,)

In [132]:
#inventory.shape
dynamic_skus.shape

(451692, 5)

In [133]:
inventory_mapping.shape

(87069, 6)

In [134]:
df_merged_4.shape

(912318, 11)

In [135]:
df_merged_5['SKU_Code_p2'].isna().sum()

0

### Stock in-hand

In [136]:
#stock in hand
stock_in_hand = df_merged_5
stock_in_hand.shape

(528839, 3)

## Returns / RTOs

In [137]:
rtos.head()

,Order Code,Order Type,Created At,Merchant,QTY,SKU QTY,Step,Status
0,Acacia-IN-472622,Inbound,2023-02-07 17:28:32.824725+00:00,Acacia,1000,1,unload,not started
1,BAS-EX-240950,Exchange,2023-02-07 16:54:12.352388+00:00,BackyardSale-SK,1,1,unload,not started
2,BAS-EX-812480,Exchange,2023-02-07 16:54:08.827443+00:00,BackyardSale-SK,1,3,unload,not started
3,BAS-EX-388241,Exchange,2023-02-07 16:54:00.555145+00:00,BackyardSale-SK,1,1,unload,not started
4,BAS-EX-812461,Exchange,2023-02-07 16:53:49.961633+00:00,BackyardSale-SK,1,1,unload,not started


In [138]:
# #Add Pending RTO
rtos['Pinding RTO']= ''
for i in range(0,len(rtos)):
    if (rtos.Step[i] == 'put_away' and rtos.Status[i] == 'completed') | (rtos.Step[i] == 'unload' and rtos.Status[i] == 'canceled') | (rtos.Step[i] == 'unload' and rtos.Status[i] == 'not started'):
        rtos['Pinding RTO'][i] = 0
    else:
        rtos['Pinding RTO'][i] = rtos['QTY'][i]

In [139]:
rtos.head()

,Order Code,Order Type,Created At,Merchant,QTY,SKU QTY,Step,Status,Pinding RTO
0,Acacia-IN-472622,Inbound,2023-02-07 17:28:32.824725+00:00,Acacia,1000,1,unload,not started,0
1,BAS-EX-240950,Exchange,2023-02-07 16:54:12.352388+00:00,BackyardSale-SK,1,1,unload,not started,0
2,BAS-EX-812480,Exchange,2023-02-07 16:54:08.827443+00:00,BackyardSale-SK,1,3,unload,not started,0
3,BAS-EX-388241,Exchange,2023-02-07 16:54:00.555145+00:00,BackyardSale-SK,1,1,unload,not started,0
4,BAS-EX-812461,Exchange,2023-02-07 16:53:49.961633+00:00,BackyardSale-SK,1,1,unload,not started,0


In [140]:
#date adjustment of rto 'Created At' Column
from dateutil import tz

# rtos['Created At']= pd.to_datetime(rtos['Created At'],format="%Y-%m-%d").dt.tz_convert(tz.tzlocal())
rtos['Created At'] = pd.to_datetime(rtos['Created At'],format="%Y-%m-%d").dt.date
rtos['Created At'] = rtos['Created At'].replace("T", " ").replace(".000Z", "")

In [141]:
rtos.head()

,Order Code,Order Type,Created At,Merchant,QTY,SKU QTY,Step,Status,Pinding RTO
0,Acacia-IN-472622,Inbound,2023-02-07,Acacia,1000,1,unload,not started,0
1,BAS-EX-240950,Exchange,2023-02-07,BackyardSale-SK,1,1,unload,not started,0
2,BAS-EX-812480,Exchange,2023-02-07,BackyardSale-SK,1,3,unload,not started,0
3,BAS-EX-388241,Exchange,2023-02-07,BackyardSale-SK,1,1,unload,not started,0
4,BAS-EX-812461,Exchange,2023-02-07,BackyardSale-SK,1,1,unload,not started,0


In [142]:
rtos.Step.unique()

array(['unload', 'put_away', 'qc', 'receiving'], dtype=object)

In [143]:
rtos_unload = rtos[rtos['Step'] != 'put_away']

In [144]:
rtos_unload.head()

,Order Code,Order Type,Created At,Merchant,QTY,SKU QTY,Step,Status,Pinding RTO
0,Acacia-IN-472622,Inbound,2023-02-07,Acacia,1000,1,unload,not started,0
1,BAS-EX-240950,Exchange,2023-02-07,BackyardSale-SK,1,1,unload,not started,0
2,BAS-EX-812480,Exchange,2023-02-07,BackyardSale-SK,1,3,unload,not started,0
3,BAS-EX-388241,Exchange,2023-02-07,BackyardSale-SK,1,1,unload,not started,0
4,BAS-EX-812461,Exchange,2023-02-07,BackyardSale-SK,1,1,unload,not started,0


In [145]:
rtos_unload.Step.unique()

array(['unload', 'qc', 'receiving'], dtype=object)

In [146]:
#replace RTOs text in order_code with OT
rtos_unload['Order Code'] = rtos_unload['Order Code'].str.replace('-RTO-','-OT-')

#if join without BAS-OT string
# rtos_unload['Order Code']  = rtos_unload['Order Code'].str.rsplit('-', 1).str[-1]   ##<-------------------------------------------------------########

rtos_unload.head()

,Order Code,Order Type,Created At,Merchant,QTY,SKU QTY,Step,Status,Pinding RTO
0,Acacia-IN-472622,Inbound,2023-02-07,Acacia,1000,1,unload,not started,0
1,BAS-EX-240950,Exchange,2023-02-07,BackyardSale-SK,1,1,unload,not started,0
2,BAS-EX-812480,Exchange,2023-02-07,BackyardSale-SK,1,3,unload,not started,0
3,BAS-EX-388241,Exchange,2023-02-07,BackyardSale-SK,1,1,unload,not started,0
4,BAS-EX-812461,Exchange,2023-02-07,BackyardSale-SK,1,1,unload,not started,0


In [147]:
rtos_unload.shape

(20249, 9)

In [148]:
#Remove Duplicates
rtos_unload = rtos_unload.drop_duplicates(subset=['Order Code'])

In [149]:
rtos_unload.shape

(20249, 9)

In [150]:
# rtos_unload_2 = rtos_unload.groupby('Order Code')['Pinding RTO'].sum().reset_index()   
rtos_unload_2 = rtos_unload.groupby(['Order Code','Created At'])['Pinding RTO'].sum().reset_index()
rtos_unload_2.head(1)

,Order Code,Created At,Pinding RTO
0,21S-EX-915307,2021-12-13,0


In [151]:
rtos_unload_2.shape

(20249, 3)

In [152]:
#if join without BAS-OT string
#df_merged_3['order_code'] = df_merged_3['order_code'].str.rsplit('KSB', 1).str[-1] ##<-------------------------------------------------------########

In [153]:
df_merged_3.head(1)

,Uniform_Date,order_type,order_id,order_code,sku_code,sku_name,merchant,merchant_name,merchant_type,industry,sku_pieces,items_cost,cash_returned,last_status,in_stock,order_date,created_at,created_date,fulfilled_on,fulfilled_date,modified_at,delivered_date,area,mapped_area,valid_area,building_no,city,mapped_city,zone_name,valid_city,valid_country,country,location_id,invoice_id,valid_phone,address_line1,valid_address_line1,is_work_address,user_id_main,customer_name,phone_number_main,courier_name,tracking_number,week_number
0,2021-02-08,outbound_order,fad5f9a2-30a2-4eda-980d-8dc6de6fe3e4,FIF-OT-FIF-1263,515KBA27,Basic Unisex Charcoal Hoodie - XL,fif,Fouad (fif),Brand,Fashion,1.0,515.0,0.0,delivered,NaN,2021-02-08,2021-02-08,2021-02-08,2021-02-09 00:00:00+00:00,2021-02-09,2021-02-08 11:42:11.249284+00:00,2021-02-10,Zamalek,Zamalek,True,4,Cairo,Cairo,cairo_giza,True,True,Egypt,113.0,11843.0,True,4 omaret al yamani Street Ground floor,True,False,1001.0,Ahmed Talaat,01001611604,Loadbugs,140981,2021-06


In [154]:
#changed order_code to order_code_new
rto_unload_3 = pd.merge(rtos_unload_2,df_merged_3, left_on='Order Code' ,right_on='order_code', how = 'left',suffixes = ("","_BKS_code"))

In [155]:
rto_unload_3.shape

(22194, 47)

In [156]:
rto_unload_3.columns

Index(['Order Code', 'Created At', 'Pinding RTO', 'Uniform_Date', 'order_type',
       'order_id', 'order_code', 'sku_code', 'sku_name', 'merchant',
       'merchant_name', 'merchant_type', 'industry', 'sku_pieces',
       'items_cost', 'cash_returned', 'last_status', 'in_stock', 'order_date',
       'created_at', 'created_date', 'fulfilled_on', 'fulfilled_date',
       'modified_at', 'delivered_date', 'area', 'mapped_area', 'valid_area',
       'building_no', 'city', 'mapped_city', 'zone_name', 'valid_city',
       'valid_country', 'country', 'location_id', 'invoice_id', 'valid_phone',
       'address_line1', 'valid_address_line1', 'is_work_address',
       'user_id_main', 'customer_name', 'phone_number_main', 'courier_name',
       'tracking_number', 'week_number'],
      dtype='object')

In [157]:
rto_unload_3.head()

,Order Code,Created At,Pinding RTO,Uniform_Date,order_type,order_id,order_code,sku_code,sku_name,merchant,merchant_name,merchant_type,industry,sku_pieces,items_cost,cash_returned,last_status,in_stock,order_date,created_at,created_date,fulfilled_on,fulfilled_date,modified_at,delivered_date,area,mapped_area,valid_area,building_no,city,mapped_city,zone_name,valid_city,valid_country,country,location_id,invoice_id,valid_phone,address_line1,valid_address_line1,is_work_address,user_id_main,customer_name,phone_number_main,courier_name,tracking_number,week_number
0,21S-EX-915307,2021-12-13,0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,21S-OT-237910,2022-02-01,0,2022-01-19,outbound_order,437fa3da-ce7f-47db-8ea8-ea7da6d3f7b3,21S-OT-237910,21utopiaM,Utopia Hoodie M,21stiches,Saleh (21stiches),Brand,Fashion,1.0,700.0,0.0,returned to origin,True,2022-01-19,2022-01-19,2022-01-19,2022-01-20 16:11:14.154524+00:00,2022-01-20,2022-01-19 21:35:45.819952+00:00,NaN,Al-Obour City,Al-Obour City,True,Villa 27/7,El Kalioubia,El Kalioubia,delta_canal,True,True,Egypt,141.0,14628.0,True,Ahmed oraby line 7,True,False,1039.0,Youssef Abo el ela,01200256113,R2S,Aa68nY1y,2022-03
2,21S-OT-303852,2022-01-23,0,2022-01-09,outbound_order,070455bb-31bf-4511-aedc-fcb7301324d0,21S-OT-303852,21donL,Don Hoodie L,21stiches,Saleh (21stiches),Brand,Fashion,1.0,2400.0,0.0,returned to origin,True,2022-01-09,2022-01-09,2022-01-09,2022-01-10 19:10:58.286922+00:00,2022-01-10,2022-01-10 08:38:47.012946+00:00,NaN,Sheikh Zayed City,Sheikh Zayed City,True,فيلا ٢٥,Giza,Giza,cairo_giza,True,True,Egypt,192.0,14217.0,True,الحي العاشر المجورة الثالثة,True,False,1039.0,Shahd Habib,01156111472,R2S,FXJTOrNN,2022-02
3,21S-OT-303852,2022-01-23,0,2022-01-09,outbound_order,070455bb-31bf-4511-aedc-fcb7301324d0,21S-OT-303852,21dondaL,Donda hoodie L,21stiches,Saleh (21stiches),Brand,Fashion,1.0,2400.0,0.0,returned to origin,True,2022-01-09,2022-01-09,2022-01-09,2022-01-10 19:10:58.286922+00:00,2022-01-10,2022-01-10 08:38:47.012946+00:00,NaN,Sheikh Zayed City,Sheikh Zayed City,True,فيلا ٢٥,Giza,Giza,cairo_giza,True,True,Egypt,192.0,14217.0,True,الحي العاشر المجورة الثالثة,True,False,1039.0,Shahd Habib,01156111472,R2S,FXJTOrNN,2022-02
4,21S-OT-303852,2022-01-23,0,2022-01-09,outbound_order,070455bb-31bf-4511-aedc-fcb7301324d0,21S-OT-303852,21nikejackL,Nike jacket L,21stiches,Saleh (21stiches),Brand,Fashion,1.0,2400.0,0.0,returned to origin,True,2022-01-09,2022-01-09,2022-01-09,2022-01-10 19:10:58.286922+00:00,2022-01-10,2022-01-10 08:38:47.012946+00:00,NaN,Sheikh Zayed City,Sheikh Zayed City,True,فيلا ٢٥,Giza,Giza,cairo_giza,True,True,Egypt,192.0,14217.0,True,الحي العاشر المجورة الثالثة,True,False,1039.0,Shahd Habib,01156111472,R2S,FXJTOrNN,2022-02


In [158]:
rto_unload_3.isna().sum()

Order Code                 0
Created At                 0
Pinding RTO                0
Uniform_Date           17034
order_type             17034
order_id               17034
order_code             17034
sku_code               17034
sku_name               17034
merchant               17034
merchant_name          17034
merchant_type          17034
industry               17034
sku_pieces             17034
items_cost             17034
cash_returned          17034
last_status            17034
in_stock               17164
order_date             17034
created_at             17034
created_date           17034
fulfilled_on           17036
fulfilled_date         17036
modified_at            17034
delivered_date         21172
area                   17043
mapped_area            17136
valid_area             17034
building_no            17358
city                   17034
mapped_city            17037
zone_name              17037
valid_city             17034
valid_country          17034
country       

In [159]:
rto_unload_3[['Order Code','order_code','Pinding RTO','sku_code']].head(2)

,Order Code,order_code,Pinding RTO,sku_code
0,21S-EX-915307,NaN,0,NaN
1,21S-OT-237910,21S-OT-237910,0,21utopiaM


In [160]:
rto_unload_3['sku_code'].notna().sum()

5160

In [161]:
rto_unload_3['sku_code'].isna().sum()

17034

In [162]:
#rto_unload_3 = rto_unload_3[['Order Code','order_code','Pinding RTO','sku_code']]
rto_unload_3 = rto_unload_3[['Order Code','order_code','Created At','Pinding RTO','sku_code']]
# rto_unload_3.head()

In [163]:
rto_unload_3 = rto_unload_3.dropna(subset=['Order Code'])

In [164]:
rto_unload_3 = rto_unload_3.dropna(subset=['sku_code'])

In [165]:
rto_unload_3.head()

,Order Code,order_code,Created At,Pinding RTO,sku_code
1,21S-OT-237910,21S-OT-237910,2022-02-01,0,21utopiaM
2,21S-OT-303852,21S-OT-303852,2022-01-23,0,21donL
3,21S-OT-303852,21S-OT-303852,2022-01-23,0,21dondaL
4,21S-OT-303852,21S-OT-303852,2022-01-23,0,21nikejackL
5,21S-OT-524121,21S-OT-524121,2022-01-23,0,21nikejackm


In [166]:
rto_unload_3.shape

(5160, 5)

In [167]:
# rto_unload_3_1 = pd.DataFrame (columns = ['BKS_sku_code','sum_sku_unload_not_started'])

In [168]:
rto_unload_3_1 = rto_unload_3.groupby(['sku_code','Created At'], dropna=False)['Pinding RTO'].sum().reset_index()

In [169]:
rto_unload_3_1.head()

,sku_code,Created At,Pinding RTO
0,#001BM,2022-08-29,0
1,(Ar01-An)Andalus,2022-01-19,0
2,0005YBL00003,2021-11-02,0
3,0005YBL00005,2022-06-16,0
4,0005YBL00006,2022-06-16,0


In [170]:
rto_unload_3_1.rename(columns = {'sku_code':'BKS_sku_code','Pinding RTO':'sum_sku_unload_not_started'}, inplace = True)
# stock in transit
rto_unload_3_1.head()

,BKS_sku_code,Created At,sum_sku_unload_not_started
0,#001BM,2022-08-29,0
1,(Ar01-An)Andalus,2022-01-19,0
2,0005YBL00003,2021-11-02,0
3,0005YBL00005,2022-06-16,0
4,0005YBL00006,2022-06-16,0


In [171]:
#inventory
stock_in_hand.head()

,SKU_Code_p2,transaction_date,QTY
0,EH-30059,2021-11-02,0
1,EH-30059,2022-04-18,1
2,EH-30059,2022-04-19,0
3,#001BL,2022-04-21,7
4,#001BL,2022-05-09,6


In [172]:
# dates_0.tail()

In [173]:
#preparing for merge for stock in hand and stock in transit
#extracting dates of stock in hand and rto_unload which is the same inv in transit

stock_in_hand['transaction_date']= pd.to_datetime(stock_in_hand['transaction_date'],format="%Y-%m-%d")
rto_unload_3_1['Created At']= pd.to_datetime(rto_unload_3_1['Created At'],format="%Y-%m-%d")

In [174]:
stock_in_hand.dtypes, rto_unload_3_1.dtypes

(SKU_Code_p2                 object
 transaction_date    datetime64[ns]
 QTY                          int64
 dtype: object,
 BKS_sku_code                          object
 Created At                    datetime64[ns]
 sum_sku_unload_not_started            object
 dtype: object)

In [175]:
#merging for both on a continous date

df_merged_6 = pd.merge(dates_0,rto_unload_3_1,left_on=['Join_Dates'],right_on=['Created At'], how = 'left',suffixes = ("","_inv"))
df_merged_6 = pd.merge(df_merged_6,stock_in_hand,left_on=['Join_Dates','BKS_sku_code'],right_on=['transaction_date','SKU_Code_p2'], how = 'outer',suffixes = ("","_inv_transit"))

df_merged_6.dropna(subset=['Created At'], inplace = True)

In [176]:
df_merged_6.shape

(4227, 7)

In [177]:
df_merged_6.head()

,Join_Dates,BKS_sku_code,Created At,sum_sku_unload_not_started,SKU_Code_p2,transaction_date,QTY
169,2021-07-27,AFR7,2021-07-27,0,AFR7,2021-07-27,0.0
170,2021-07-27,BLK7,2021-07-27,0,BLK7,2021-07-27,0.0
171,2021-07-27,quiltedw8,2021-07-27,0,NaN,NaT,NaN
175,2021-07-31,FWRUC20Y,2021-07-31,0,NaN,NaT,NaN
180,2021-08-05,12,2021-08-05,0,NaN,NaT,NaN


In [178]:
df_merged_6.drop(['Created At','transaction_date'], inplace = True, axis = 1)

In [179]:
df_merged_6.head()

,Join_Dates,BKS_sku_code,sum_sku_unload_not_started,SKU_Code_p2,QTY
169,2021-07-27,AFR7,0,AFR7,0.0
170,2021-07-27,BLK7,0,BLK7,0.0
171,2021-07-27,quiltedw8,0,NaN,NaN
175,2021-07-31,FWRUC20Y,0,NaN,NaN
180,2021-08-05,12,0,NaN,NaN


In [180]:
# df_merged_6['sum_sku_unload_not_started'] [np.isnan(df_merged_6['sum_sku_unload_not_started'])]=0 
df_merged_6['sum_sku_unload_not_started'] [pd.isnull(df_merged_6['sum_sku_unload_not_started'])]=0 

# df_merged_6['sum_sku_unload_not_started'] 
df_merged_6

,Join_Dates,BKS_sku_code,sum_sku_unload_not_started,SKU_Code_p2,QTY
169,2021-07-27,AFR7,0,AFR7,0.0
170,2021-07-27,BLK7,0,BLK7,0.0
171,2021-07-27,quiltedw8,0,NaN,NaN
175,2021-07-31,FWRUC20Y,0,NaN,NaN
180,2021-08-05,12,0,NaN,NaN
...,...,...,...,...,...
4565,2023-02-07,زيت مهابهارينج 100 مل,6,زيت مهابهارينج 100 مل,65.0
4566,2023-02-07,شامبو كيش كينج العادى 200 مل,17,شامبو كيش كينج العادى 200 مل,235.0
4567,2023-02-07,شنطة تغليف,6,شنطة تغليف,76.0
4568,2023-02-07,موديل ساره شتوي اسود XXL,1,NaN,NaN


In [181]:
df_merged_6.shape

(4227, 5)

In [182]:
df_merged_6['sum_sku_unload_not_started'].isna().sum()

0

In [183]:
df_merged_6.isna().sum()

Join_Dates                       0
BKS_sku_code                     0
sum_sku_unload_not_started       0
SKU_Code_p2                   2432
QTY                           2432
dtype: int64

In [184]:
#forward full for quantities of SKUS in hand until value changes
df_merged_6['QTY'] = df_merged_6['QTY'].ffill(axis = 0)

In [185]:
df_merged_6

,Join_Dates,BKS_sku_code,sum_sku_unload_not_started,SKU_Code_p2,QTY
169,2021-07-27,AFR7,0,AFR7,0.0
170,2021-07-27,BLK7,0,BLK7,0.0
171,2021-07-27,quiltedw8,0,NaN,0.0
175,2021-07-31,FWRUC20Y,0,NaN,0.0
180,2021-08-05,12,0,NaN,0.0
...,...,...,...,...,...
4565,2023-02-07,زيت مهابهارينج 100 مل,6,زيت مهابهارينج 100 مل,65.0
4566,2023-02-07,شامبو كيش كينج العادى 200 مل,17,شامبو كيش كينج العادى 200 مل,235.0
4567,2023-02-07,شنطة تغليف,6,شنطة تغليف,76.0
4568,2023-02-07,موديل ساره شتوي اسود XXL,1,NaN,76.0


In [186]:
df_merged_6.isna().sum()

Join_Dates                       0
BKS_sku_code                     0
sum_sku_unload_not_started       0
SKU_Code_p2                   2432
QTY                              0
dtype: int64

In [187]:
# df_merged_7 = df_merged_6[['SKU_Code_p2','Join_Dates','QTY','sum_sku_unload_not_started']]
df_merged_7 = df_merged_6[['BKS_sku_code','Join_Dates','QTY','sum_sku_unload_not_started']]


In [188]:
df_merged_7.head()

,BKS_sku_code,Join_Dates,QTY,sum_sku_unload_not_started
169,AFR7,2021-07-27,0.0,0
170,BLK7,2021-07-27,0.0,0
171,quiltedw8,2021-07-27,0.0,0
175,FWRUC20Y,2021-07-31,0.0,0
180,12,2021-08-05,0.0,0


## Total Inventory

In [189]:
#total inventory
df_merged_7

,BKS_sku_code,Join_Dates,QTY,sum_sku_unload_not_started
169,AFR7,2021-07-27,0.0,0
170,BLK7,2021-07-27,0.0,0
171,quiltedw8,2021-07-27,0.0,0
175,FWRUC20Y,2021-07-31,0.0,0
180,12,2021-08-05,0.0,0
...,...,...,...,...
4565,زيت مهابهارينج 100 مل,2023-02-07,65.0,6
4566,شامبو كيش كينج العادى 200 مل,2023-02-07,235.0,17
4567,شنطة تغليف,2023-02-07,76.0,6
4568,موديل ساره شتوي اسود XXL,2023-02-07,76.0,1


In [190]:
df_merged_7.isna().sum()

BKS_sku_code                  0
Join_Dates                    0
QTY                           0
sum_sku_unload_not_started    0
dtype: int64

In [191]:
## Order Volumes

# df_merged_3[['order_code', 'last_status', 'sku_code', 'sku_pieces', 'customer_name', 'city', 'area','items_cost']]
df_merged_3

,Uniform_Date,order_type,order_id,order_code,sku_code,sku_name,merchant,merchant_name,merchant_type,industry,sku_pieces,items_cost,cash_returned,last_status,in_stock,order_date,created_at,created_date,fulfilled_on,fulfilled_date,modified_at,delivered_date,area,mapped_area,valid_area,building_no,city,mapped_city,zone_name,valid_city,valid_country,country,location_id,invoice_id,valid_phone,address_line1,valid_address_line1,is_work_address,user_id_main,customer_name,phone_number_main,courier_name,tracking_number,week_number
0,2021-02-08,outbound_order,fad5f9a2-30a2-4eda-980d-8dc6de6fe3e4,FIF-OT-FIF-1263,515KBA27,Basic Unisex Charcoal Hoodie - XL,fif,Fouad (fif),Brand,Fashion,1.0,515.0,0.0,delivered,NaN,2021-02-08,2021-02-08,2021-02-08,2021-02-09 00:00:00+00:00,2021-02-09,2021-02-08 11:42:11.249284+00:00,2021-02-10,Zamalek,Zamalek,True,4,Cairo,Cairo,cairo_giza,True,True,Egypt,113.0,11843.0,True,4 omaret al yamani Street Ground floor,True,False,1001.0,Ahmed Talaat,01001611604,Loadbugs,140981,2021-06
1,2021-02-08,outbound_order,63496d77-86d1-4345-8a46-4400a0f21a38,FIF-OT-FIF-2000,FIHBXSW,Beirut Hoodie-XS-White,fif,Fouad (fif),Brand,Fashion,1.0,0.0,0.0,delivered,NaN,2021-02-08,2021-02-08,2021-02-08,2021-02-09 00:00:00+00:00,2021-02-09,2021-02-08 11:50:42.125285+00:00,2021-02-10,Nasr City,Nasr City,True,24,Cairo,Cairo,cairo_giza,True,True,Egypt,104.0,11843.0,True,24 Ahmed Fakhry Street,True,False,1001.0,Sarah Mohamed,01286590341,Loadbugs,140982,2021-06
2,2021-02-08,outbound_order,63496d77-86d1-4345-8a46-4400a0f21a38,FIF-OT-FIF-2000,FISLWLG,Lil Wayne Sweater-L-Grey,fif,Fouad (fif),Brand,Fashion,1.0,0.0,0.0,delivered,NaN,2021-02-08,2021-02-08,2021-02-08,2021-02-09 00:00:00+00:00,2021-02-09,2021-02-08 11:50:42.125285+00:00,2021-02-10,Nasr City,Nasr City,True,24,Cairo,Cairo,cairo_giza,True,True,Egypt,104.0,11843.0,True,24 Ahmed Fakhry Street,True,False,1001.0,Sarah Mohamed,01286590341,Loadbugs,140982,2021-06
3,2021-02-08,outbound_order,b53b99fe-a36e-47be-a5b9-ed3e91109e2a,HIB-OT-401628,CMH001,HiBrew Coffee Machine,hibrew,HiBREW (hibrew),Brand,Home,1.0,0.0,0.0,pick up,NaN,2021-02-08,2021-02-08,2021-02-08,2021-02-08 00:00:00+00:00,2021-02-08,2021-07-07 16:17:43.511658+00:00,NaN,3rd Settlement,NaN,False,Warehouse Pickup,Cairo,Cairo,cairo_giza,True,True,Egypt,NaN,11863.0,True,Warehouse Pickup,False,False,1004.0,David Azmy,01234567890,Flextock (Same day),Pick up,2021-06
4,2021-02-08,outbound_order,05e3f750-4fa1-4c9e-8b78-1f15d630af0e,FIF-OT-FIF-1252_1,3H6F4MB2,FIF Basic Unisex Black On Black Sweatpants Gre...,fif,Fouad (fif),Brand,Fashion,1.0,0.0,0.0,delivered,NaN,2021-02-08,2021-02-08,2021-02-08,2021-02-10 00:00:00+00:00,2021-02-10,2021-02-08 13:24:56.982701+00:00,2021-02-11,Heliopolis,Heliopolis,True,97,Cairo,Cairo,cairo_giza,True,True,Egypt,99.0,11843.0,True,97 elmarghany street masr el gdeda,True,False,1001.0,Omar Mahsoub,01005332092,Loadbugs,141086,2021-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1024896,2023-02-07,outbound_order,f3824c48-7e1c-4b49-95dd-7429bdfc66c0,BND-OT-6280,10131038,Be-Strong Blue Tie-dye - 38,Beindie,Farah (Beindie),Brand,Fashion,1.0,1300.0,0.0,pending,True,2023-02-07,2023-02-07,2023-02-07,NaN,NaN,2023-02-07 22:27:36.048062+00:00,NaN,New Cairo,New Cairo,True,0,Cairo,Cairo,cairo_giza,True,True,Egypt,105.0,NaN,True,Compound Lakeview,True,False,1628.0,Emmy Tarek,01018848473,NaN,NaN,2023-06
1024897,2023-02-07,outbound_order,205151a1-7cb6-4bc6-b0b1-cdb188240501,ZHO-OT-66148,101,السبيرولينا,Zohor,Mohamed Marzok(Zohor),Brand,Trendy Products,1.0,345.0,0.0,processing,True,2023-02-07,2023-02-07,2023-02-07,NaN,NaN,2023-02-07 10:10:35.900169+00:00,NaN,الجيزة,Agouza,True,1,الهرم,Giza,cairo_giza,True,True,Egypt,168.0,NaN,True,محافظة الجيزة - الهرم - ٨٣ شارع الهرم - فوق نف...,True,False,1269.0,NaN,01007912692,JT Express,JEG000033457762,2023-06
1024898,2023-02-07,outbound_order,63e0eb01-105e-4fb4

In [192]:
# manual_input

### Combining Inventory on hand, inventory in transit (rto unloaded) and Volume Ordered (Demand)

In [193]:
#df_merged_3 is order volumes
pd.get_option("display.max_columns")
df_merged_3.head(2)

,Uniform_Date,order_type,order_id,order_code,sku_code,sku_name,merchant,merchant_name,merchant_type,industry,sku_pieces,items_cost,cash_returned,last_status,in_stock,order_date,created_at,created_date,fulfilled_on,fulfilled_date,modified_at,delivered_date,area,mapped_area,valid_area,building_no,city,mapped_city,zone_name,valid_city,valid_country,country,location_id,invoice_id,valid_phone,address_line1,valid_address_line1,is_work_address,user_id_main,customer_name,phone_number_main,courier_name,tracking_number,week_number
0,2021-02-08,outbound_order,fad5f9a2-30a2-4eda-980d-8dc6de6fe3e4,FIF-OT-FIF-1263,515KBA27,Basic Unisex Charcoal Hoodie - XL,fif,Fouad (fif),Brand,Fashion,1.0,515.0,0.0,delivered,NaN,2021-02-08,2021-02-08,2021-02-08,2021-02-09 00:00:00+00:00,2021-02-09,2021-02-08 11:42:11.249284+00:00,2021-02-10,Zamalek,Zamalek,True,4,Cairo,Cairo,cairo_giza,True,True,Egypt,113.0,11843.0,True,4 omaret al yamani Street Ground floor,True,False,1001.0,Ahmed Talaat,01001611604,Loadbugs,140981,2021-06
1,2021-02-08,outbound_order,63496d77-86d1-4345-8a46-4400a0f21a38,FIF-OT-FIF-2000,FIHBXSW,Beirut Hoodie-XS-White,fif,Fouad (fif),Brand,Fashion,1.0,0.0,0.0,delivered,NaN,2021-02-08,2021-02-08,2021-02-08,2021-02-09 00:00:00+00:00,2021-02-09,2021-02-08 11:50:42.125285+00:00,2021-02-10,Nasr City,Nasr City,True,24,Cairo,Cairo,cairo_giza,True,True,Egypt,104.0,11843.0,True,24 Ahmed Fakhry Street,True,False,1001.0,Sarah Mohamed,01286590341,Loadbugs,140982,2021-06


In [194]:
df_merged_3.merchant_type.unique()
df_merged_3.isna().sum()


Uniform_Date                0
order_type                  1
order_id                    1
order_code                  1
sku_code                    1
sku_name                    1
merchant                    1
merchant_name               1
merchant_type               1
industry                    1
sku_pieces                  1
items_cost                  1
cash_returned               1
last_status                 1
in_stock                50188
order_date                  1
created_at                  1
created_date                1
fulfilled_on            11913
fulfilled_date          11913
modified_at                 1
delivered_date         188064
area                     1834
mapped_area             10839
valid_area                  1
building_no             15232
city                      446
mapped_city               539
zone_name                 539
valid_city                  1
valid_country               1
country                   311
location_id             10839
invoice_id

In [195]:
df_merged_3.columns

Index(['Uniform_Date', 'order_type', 'order_id', 'order_code', 'sku_code',
       'sku_name', 'merchant', 'merchant_name', 'merchant_type', 'industry',
       'sku_pieces', 'items_cost', 'cash_returned', 'last_status', 'in_stock',
       'order_date', 'created_at', 'created_date', 'fulfilled_on',
       'fulfilled_date', 'modified_at', 'delivered_date', 'area',
       'mapped_area', 'valid_area', 'building_no', 'city', 'mapped_city',
       'zone_name', 'valid_city', 'valid_country', 'country', 'location_id',
       'invoice_id', 'valid_phone', 'address_line1', 'valid_address_line1',
       'is_work_address', 'user_id_main', 'customer_name', 'phone_number_main',
       'courier_name', 'tracking_number', 'week_number'],
      dtype='object')

In [269]:
### Pulling relevant columns for ordered volumes

In [196]:
#adding order code back in to see if acive sellers needs to be added back from new active seller files
df_merged_3.rename(columns= {'Created Date': 'created_date'}, inplace = True)
ordered_volumes = df_merged_3[['order_code','order_date','created_date','sku_code','sku_pieces','merchant_type', 'merchant','customer_name','industry','week_number', 'items_cost','in_stock']]
ordered_volumes.head()

,order_code,order_date,created_date,sku_code,sku_pieces,merchant_type,merchant,customer_name,industry,week_number,items_cost,in_stock
0,FIF-OT-FIF-1263,2021-02-08,2021-02-08,515KBA27,1.0,Brand,fif,Ahmed Talaat,Fashion,2021-06,515.0,NaN
1,FIF-OT-FIF-2000,2021-02-08,2021-02-08,FIHBXSW,1.0,Brand,fif,Sarah Mohamed,Fashion,2021-06,0.0,NaN
2,FIF-OT-FIF-2000,2021-02-08,2021-02-08,FISLWLG,1.0,Brand,fif,Sarah Mohamed,Fashion,2021-06,0.0,NaN
3,HIB-OT-401628,2021-02-08,2021-02-08,CMH001,1.0,Brand,hibrew,David Azmy,Home,2021-06,0.0,NaN
4,FIF-OT-FIF-1252_1,2021-02-08,2021-02-08,3H6F4MB2,1.0,Brand,fif,Omar Mahsoub,Fashion,2021-06,0.0,NaN


In [197]:
#drop Null rows or Null order_codes
ordered_volumes.dropna(axis = 0, how = 'all', inplace = True)

In [198]:
ordered_volumes.tail(2)

,order_code,order_date,created_date,sku_code,sku_pieces,merchant_type,merchant,customer_name,industry,week_number,items_cost,in_stock
1024899,SHC-OT-12485,2023-02-07,2023-02-07,MB049201,1.0,Brand,SohatiCare,Mary Wageh,Cosmetics,2023-06,593.1,True
1024900,ZHO-OT-66073,2023-02-06,2023-02-07,406,1.0,Brand,Zohor,NaN,Trendy Products,2023-06,345.0,True


In [199]:
# ordered_volumes[ordered_volumes.in_stock == False]
ordered_volumes[ordered_volumes.in_stock.isnull()].order_date.min()

'2013-07-21'

In [200]:
df_merged_7

,BKS_sku_code,Join_Dates,QTY,sum_sku_unload_not_started
169,AFR7,2021-07-27,0.0,0
170,BLK7,2021-07-27,0.0,0
171,quiltedw8,2021-07-27,0.0,0
175,FWRUC20Y,2021-07-31,0.0,0
180,12,2021-08-05,0.0,0
...,...,...,...,...
4565,زيت مهابهارينج 100 مل,2023-02-07,65.0,6
4566,شامبو كيش كينج العادى 200 مل,2023-02-07,235.0,17
4567,شنطة تغليف,2023-02-07,76.0,6
4568,موديل ساره شتوي اسود XXL,2023-02-07,76.0,1


In [270]:
### Setting inventory volumes columns

In [201]:
df_merged_7.rename(columns = {'BKS_sku_code':'sku_code','QTY':'inv_on_hand','sum_sku_unload_not_started':'inv_in_transit'}, inplace = True)
inventory_volumes = df_merged_7
inventory_volumes.head()

,sku_code,Join_Dates,inv_on_hand,inv_in_transit
169,AFR7,2021-07-27,0.0,0
170,BLK7,2021-07-27,0.0,0
171,quiltedw8,2021-07-27,0.0,0
175,FWRUC20Y,2021-07-31,0.0,0
180,12,2021-08-05,0.0,0


In [202]:
inventory_volumes.isna().sum()

sku_code          0
Join_Dates        0
inv_on_hand       0
inv_in_transit    0
dtype: int64

In [271]:
### Merge for ordered volume and inventory volume

In [203]:
ordered_volumes.isna().sum()

order_code           0
order_date           0
created_date         0
sku_code             0
sku_pieces           0
merchant_type        0
merchant             0
customer_name    29756
industry             0
week_number          0
items_cost           0
in_stock         50187
dtype: int64

In [204]:
ordered_volumes.shape

(1024900, 12)

In [205]:
ordered_volumes['created_date'] = pd.to_datetime(ordered_volumes['created_date'])
ordered_volumes.dtypes

order_code               object
order_date               object
created_date     datetime64[ns]
sku_code                 object
sku_pieces              float64
merchant_type            object
merchant                 object
customer_name            object
industry                 object
week_number              object
items_cost              float64
in_stock                 object
dtype: object

In [206]:
ordered_volumes = pd.merge(dates_0,ordered_volumes,left_on=['Join_Dates'],right_on=['created_date'], how = 'left')
ordered_volumes.head()

,Join_Dates,order_code,order_date,created_date,sku_code,sku_pieces,merchant_type,merchant,customer_name,industry,week_number,items_cost,in_stock
0,2021-02-08,FIF-OT-FIF-1263,2021-02-08,2021-02-08,515KBA27,1.0,Brand,fif,Ahmed Talaat,Fashion,2021-06,515.0,NaN
1,2021-02-08,FIF-OT-FIF-2000,2021-02-08,2021-02-08,FIHBXSW,1.0,Brand,fif,Sarah Mohamed,Fashion,2021-06,0.0,NaN
2,2021-02-08,FIF-OT-FIF-2000,2021-02-08,2021-02-08,FISLWLG,1.0,Brand,fif,Sarah Mohamed,Fashion,2021-06,0.0,NaN
3,2021-02-08,HIB-OT-401628,2021-02-08,2021-02-08,CMH001,1.0,Brand,hibrew,David Azmy,Home,2021-06,0.0,NaN
4,2021-02-08,FIF-OT-FIF-1252_1,2021-02-08,2021-02-08,3H6F4MB2,1.0,Brand,fif,Omar Mahsoub,Fashion,2021-06,0.0,NaN


In [207]:
dataset_0 = pd.merge(ordered_volumes,inventory_volumes, left_on=['Join_Dates','sku_code'] ,right_on=['Join_Dates','sku_code'], how = 'left',suffixes = ("_order","_inv"))
dataset_0.head()

,Join_Dates,order_code,order_date,created_date,sku_code,sku_pieces,merchant_type,merchant,customer_name,industry,week_number,items_cost,in_stock,inv_on_hand,inv_in_transit
0,2021-02-08,FIF-OT-FIF-1263,2021-02-08,2021-02-08,515KBA27,1.0,Brand,fif,Ahmed Talaat,Fashion,2021-06,515.0,NaN,NaN,NaN
1,2021-02-08,FIF-OT-FIF-2000,2021-02-08,2021-02-08,FIHBXSW,1.0,Brand,fif,Sarah Mohamed,Fashion,2021-06,0.0,NaN,NaN,NaN
2,2021-02-08,FIF-OT-FIF-2000,2021-02-08,2021-02-08,FISLWLG,1.0,Brand,fif,Sarah Mohamed,Fashion,2021-06,0.0,NaN,NaN,NaN
3,2021-02-08,HIB-OT-401628,2021-02-08,2021-02-08,CMH001,1.0,Brand,hibrew,David Azmy,Home,2021-06,0.0,NaN,NaN,NaN
4,2021-02-08,FIF-OT-FIF-1252_1,2021-02-08,2021-02-08,3H6F4MB2,1.0,Brand,fif,Omar Mahsoub,Fashion,2021-06,0.0,NaN,NaN,NaN


In [208]:
dataset_0.tail(2)

,Join_Dates,order_code,order_date,created_date,sku_code,sku_pieces,merchant_type,merchant,customer_name,industry,week_number,items_cost,in_stock,inv_on_hand,inv_in_transit
1024899,2023-02-07,SHC-OT-12485,2023-02-07,2023-02-07,MB049201,1.0,Brand,SohatiCare,Mary Wageh,Cosmetics,2023-06,593.1,True,NaN,NaN
1024900,2023-02-07,ZHO-OT-66073,2023-02-06,2023-02-07,406,1.0,Brand,Zohor,NaN,Trendy Products,2023-06,345.0,True,5410.0,3


In [209]:
# dataset_0.dropna(subset = ['inv_on_hand','inv_in_transit'], inplace = True)
# dataset_0.head()

In [210]:
dataset_0.shape

(1024901, 15)

In [211]:
dataset_0 = dataset_0.dropna(subset = ['order_date','created_date'])

In [212]:
dataset_0.isna().sum()

Join_Dates             0
order_code             0
order_date             0
created_date           0
sku_code               0
sku_pieces             0
merchant_type          0
merchant               0
customer_name      29756
industry               0
week_number            0
items_cost             0
in_stock           50187
inv_on_hand       994656
inv_in_transit    994656
dtype: int64

In [213]:
dataset_0.head()

,Join_Dates,order_code,order_date,created_date,sku_code,sku_pieces,merchant_type,merchant,customer_name,industry,week_number,items_cost,in_stock,inv_on_hand,inv_in_transit
0,2021-02-08,FIF-OT-FIF-1263,2021-02-08,2021-02-08,515KBA27,1.0,Brand,fif,Ahmed Talaat,Fashion,2021-06,515.0,NaN,NaN,NaN
1,2021-02-08,FIF-OT-FIF-2000,2021-02-08,2021-02-08,FIHBXSW,1.0,Brand,fif,Sarah Mohamed,Fashion,2021-06,0.0,NaN,NaN,NaN
2,2021-02-08,FIF-OT-FIF-2000,2021-02-08,2021-02-08,FISLWLG,1.0,Brand,fif,Sarah Mohamed,Fashion,2021-06,0.0,NaN,NaN,NaN
3,2021-02-08,HIB-OT-401628,2021-02-08,2021-02-08,CMH001,1.0,Brand,hibrew,David Azmy,Home,2021-06,0.0,NaN,NaN,NaN
4,2021-02-08,FIF-OT-FIF-1252_1,2021-02-08,2021-02-08,3H6F4MB2,1.0,Brand,fif,Omar Mahsoub,Fashion,2021-06,0.0,NaN,NaN,NaN


In [214]:
dataset_0.tail(2)

,Join_Dates,order_code,order_date,created_date,sku_code,sku_pieces,merchant_type,merchant,customer_name,industry,week_number,items_cost,in_stock,inv_on_hand,inv_in_transit
1024899,2023-02-07,SHC-OT-12485,2023-02-07,2023-02-07,MB049201,1.0,Brand,SohatiCare,Mary Wageh,Cosmetics,2023-06,593.1,True,NaN,NaN
1024900,2023-02-07,ZHO-OT-66073,2023-02-06,2023-02-07,406,1.0,Brand,Zohor,NaN,Trendy Products,2023-06,345.0,True,5410.0,3


In [215]:
dataset_0.shape

(1024900, 15)

Changes start (here) since order_date of merchant outbounds shifted to order table created_at

In [216]:
# dataset_0.to_excel('file00.xlsx')

In [217]:
dataset_0.drop(['order_date'], axis = 1, inplace = True)
dataset_0.rename(columns = {'created_date':'order_date'}, inplace = True)
dataset_0.head()

,Join_Dates,order_code,order_date,sku_code,sku_pieces,merchant_type,merchant,customer_name,industry,week_number,items_cost,in_stock,inv_on_hand,inv_in_transit
0,2021-02-08,FIF-OT-FIF-1263,2021-02-08,515KBA27,1.0,Brand,fif,Ahmed Talaat,Fashion,2021-06,515.0,NaN,NaN,NaN
1,2021-02-08,FIF-OT-FIF-2000,2021-02-08,FIHBXSW,1.0,Brand,fif,Sarah Mohamed,Fashion,2021-06,0.0,NaN,NaN,NaN
2,2021-02-08,FIF-OT-FIF-2000,2021-02-08,FISLWLG,1.0,Brand,fif,Sarah Mohamed,Fashion,2021-06,0.0,NaN,NaN,NaN
3,2021-02-08,HIB-OT-401628,2021-02-08,CMH001,1.0,Brand,hibrew,David Azmy,Home,2021-06,0.0,NaN,NaN,NaN
4,2021-02-08,FIF-OT-FIF-1252_1,2021-02-08,3H6F4MB2,1.0,Brand,fif,Omar Mahsoub,Fashion,2021-06,0.0,NaN,NaN,NaN


In [218]:
# from datetime import datetime, timedelta

# def from_excel_ordinal(ordinal: float, _epoch0=datetime(2020, 12, 31)) -> datetime:
#     if ordinal >= 60:
#         ordinal -= 1  # Excel leap year bug, 1900 is not a leap year!
#     return (_epoch0 + timedelta(days=ordinal)).replace(microsecond=0)

# m= dataset_0['order_date'].astype(str).str.isdigit()

# dataset_0.loc[m, 'order_date'] = dataset_0.loc[m, 'order_date'].astype(int).apply(from_excel_ordinal)
# dataset_0['order_date'] = pd.to_datetime(dataset_0['order_date'], errors='coerce')



In [219]:
#Drop for nans (required only after ffill for inventory and zeros for intransit)
# dataset_0['order_date'] = dataset_0['order_date'].astype(str)
# pd.to_datetime(dataset_0['order_date'].astype(str), format = '%d/%m/%Y %H:%M:%S')

In [220]:
dataset_0.dropna(axis=0, how='all')
dataset_0.dropna(axis=0, subset = ['sku_code'])
dataset_0.shape

(1024900, 14)

In [221]:
dataset_0.order_date.min()

Timestamp('2021-02-08 00:00:00')

In [222]:
dataset_0.order_date.max()

Timestamp('2023-02-07 00:00:00')

In [223]:
dataset_0.dtypes

Join_Dates        datetime64[ns]
order_code                object
order_date        datetime64[ns]
sku_code                  object
sku_pieces               float64
merchant_type             object
merchant                  object
customer_name             object
industry                  object
week_number               object
items_cost               float64
in_stock                  object
inv_on_hand              float64
inv_in_transit            object
dtype: object

In [224]:
# dataset_0.to_excel('file011.xlsx')
# dataset_0.tail(1)

In [225]:
dataset_0.shape

(1024900, 14)

In [226]:
dataset_0.isna().sum()
# dataset_0.dtypes

Join_Dates             0
order_code             0
order_date             0
sku_code               0
sku_pieces             0
merchant_type          0
merchant               0
customer_name      29756
industry               0
week_number            0
items_cost             0
in_stock           50187
inv_on_hand       994656
inv_in_transit    994656
dtype: int64

In [227]:
# manual_input.head()

In [228]:
# dtype = pd.SparseDtype(np.dtype('datetime64[ns]'))
# series = pd.Series(dataset_0['order_date'], dtype=dtype)
# dataset_0['order_date']=np.array(series)

dataset_0['order_date'] = pd.to_datetime(dataset_0['order_date'], errors = 'coerce')

In [229]:
dataset_0.shape
dataset_0.isna().sum()

Join_Dates             0
order_code             0
order_date             0
sku_code               0
sku_pieces             0
merchant_type          0
merchant               0
customer_name      29756
industry               0
week_number            0
items_cost             0
in_stock           50187
inv_on_hand       994656
inv_in_transit    994656
dtype: int64

In [230]:
#Adding year and month column in each datasets to merge on categories as days dont match for manual input file
dataset_0['Year']=dataset_0['order_date'].dt.year
dataset_0['month']=dataset_0['order_date'].dt.month
# dataset_0.head()

In [231]:
dataset= dataset_0.copy()
dataset.isna().sum()

Join_Dates             0
order_code             0
order_date             0
sku_code               0
sku_pieces             0
merchant_type          0
merchant               0
customer_name      29756
industry               0
week_number            0
items_cost             0
in_stock           50187
inv_on_hand       994656
inv_in_transit    994656
Year                   0
month                  0
dtype: int64

In [232]:
dataset['sku_code'].nunique()

30685

In [233]:
dataset.dtypes

Join_Dates        datetime64[ns]
order_code                object
order_date        datetime64[ns]
sku_code                  object
sku_pieces               float64
merchant_type             object
merchant                  object
customer_name             object
industry                  object
week_number               object
items_cost               float64
in_stock                  object
inv_on_hand              float64
inv_in_transit            object
Year                       int64
month                      int64
dtype: object

## Demand and Inventory 
### Demand = Orders (SKU Pieces Ordered)
### Inventory = On Hand + in Transit

In [240]:
dataset_4 = dataset.copy()

#zero for nan values in skus
dataset_4[['sku_pieces','inv_on_hand','inv_in_transit']] =dataset_4[['sku_pieces','inv_on_hand','inv_in_transit']].fillna(0)
#sum for inventory
dataset_4['Total_Inv'] = dataset_4[['inv_on_hand','inv_in_transit']].sum(axis =1)


In [241]:
dataset_4.head()

,Join_Dates,order_code,order_date,sku_code,sku_pieces,merchant_type,merchant,customer_name,industry,week_number,items_cost,in_stock,inv_on_hand,inv_in_transit,Year,month,Total_Inv
0,2021-02-08,FIF-OT-FIF-1263,2021-02-08,515KBA27,1.0,Brand,fif,Ahmed Talaat,Fashion,2021-06,515.0,NaN,0.0,0,2021,2,0.0
1,2021-02-08,FIF-OT-FIF-2000,2021-02-08,FIHBXSW,1.0,Brand,fif,Sarah Mohamed,Fashion,2021-06,0.0,NaN,0.0,0,2021,2,0.0
2,2021-02-08,FIF-OT-FIF-2000,2021-02-08,FISLWLG,1.0,Brand,fif,Sarah Mohamed,Fashion,2021-06,0.0,NaN,0.0,0,2021,2,0.0
3,2021-02-08,HIB-OT-401628,2021-02-08,CMH001,1.0,Brand,hibrew,David Azmy,Home,2021-06,0.0,NaN,0.0,0,2021,2,0.0
4,2021-02-08,FIF-OT-FIF-1252_1,2021-02-08,3H6F4MB2,1.0,Brand,fif,Omar Mahsoub,Fashion,2021-06,0.0,NaN,0.0,0,2021,2,0.0


In [242]:
dataset_4.isna().sum()

Join_Dates            0
order_code            0
order_date            0
sku_code              0
sku_pieces            0
merchant_type         0
merchant              0
customer_name     29756
industry              0
week_number           0
items_cost            0
in_stock          50187
inv_on_hand           0
inv_in_transit        0
Year                  0
month                 0
Total_Inv             0
dtype: int64

In [243]:
dates = dates_0.copy()
dates.columns = ['Uniform_Date']
dates.head()

,Uniform_Date
0,2021-02-08
1,2021-02-09
2,2021-02-10
3,2021-02-11
4,2021-02-12


In [244]:
dataset_4.shape

(1024900, 17)

In [245]:
dataset_4_1 =  pd.merge(dates,dataset_4, left_on=['Uniform_Date'],right_on=['order_date'], how = 'left',suffixes = ("","_"))
# pd.merge_asof(dates,dataset_4, left_on=['join_date'],right_on=['closest_date'])
# tmp = pd.merge_asof(dates,dataset_4,left_on='Uniform_Date',right_on='closest_date',direction='nearest', tolerance=pd.Timedelta("7d"),allow_exact_matches=False)
# tmp

dataset_4_1.head(20)

,Uniform_Date,Join_Dates,order_code,order_date,sku_code,sku_pieces,merchant_type,merchant,customer_name,industry,week_number,items_cost,in_stock,inv_on_hand,inv_in_transit,Year,month,Total_Inv
0,2021-02-08,2021-02-08,FIF-OT-FIF-1263,2021-02-08,515KBA27,1.0,Brand,fif,Ahmed Talaat,Fashion,2021-06,515.0,NaN,0.0,0.0,2021.0,2.0,0.0
1,2021-02-08,2021-02-08,FIF-OT-FIF-2000,2021-02-08,FIHBXSW,1.0,Brand,fif,Sarah Mohamed,Fashion,2021-06,0.0,NaN,0.0,0.0,2021.0,2.0,0.0
2,2021-02-08,2021-02-08,FIF-OT-FIF-2000,2021-02-08,FISLWLG,1.0,Brand,fif,Sarah Mohamed,Fashion,2021-06,0.0,NaN,0.0,0.0,2021.0,2.0,0.0
3,2021-02-08,2021-02-08,HIB-OT-401628,2021-02-08,CMH001,1.0,Brand,hibrew,David Azmy,Home,2021-06,0.0,NaN,0.0,0.0,2021.0,2.0,0.0
4,2021-02-08,2021-02-08,FIF-OT-FIF-1252_1,2021-02-08,3H6F4MB2,1.0,Brand,fif,Omar Mahsoub,Fashion,2021-06,0.0,NaN,0.0,0.0,2021.0,2.0,0.0
5,2021-02-08,2021-02-08,HIB-OT-HiBrew 41,2021-02-08,MPOT21,1.0,Brand,hibrew,Ayman Gabr,Home,2021-06,1245.0,NaN,0.0,0.0,2021.0,2.0,0.0
6,2021-02-08,2021-02-08,HIB-OT-HiBrew 42,2021-02-08,MPOT21,1.0,Brand,hibrew,Niveen Nazeer,Home,2021-06,1245.0,NaN,0.0,0.0,2021.0,2.0,0.0
7,2021-02-08,2021-02-08,FIF-OT-FIF-1264,2021-02-08,LWLQA242,1.0,Brand,fif,Abdelrahman Hany,Fashion,2021-06,465.0,NaN,0.0,0.0,2021.0,2.0,0.0
8,2021-02-08,2021-02-08,FIF-OT-FIF-1262,2021-02-08,31RL9YTE,1.0,Brand,fif,Sherine Fawzy,Fashion,2021-06,565.0,NaN,0.0,0.0,2021.0,2.0,0.0
9,2021-02-09,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [246]:
dataset_4_1.isna().sum()

Uniform_Date          0
Join_Dates            1
order_code            1
order_date            1
sku_code              1
sku_pieces            1
merchant_type         1
merchant              1
customer_name     29757
industry              1
week_number           1
items_cost            1
in_stock          50188
inv_on_hand           1
inv_in_transit        1
Year                  1
month                 1
Total_Inv             1
dtype: int64

In [247]:
dataset_4_1['Uniform_Date'].nunique()

730

In [248]:
main_dataset = dataset_4_1.copy() 
# main_dataset

In [249]:
main_dataset['Uniform_Date'].nunique()

730

### Add for Active Sellers (requested post-dataset creation by finance)

In [250]:
# alkaseeba_active_seller['Created At']= pd.to_datetime(alkaseeba_active_seller['Created At'],format="%Y-%m-%d").dt.date
# alkaseeba_active_seller['Created At'] = alkaseeba_active_seller['Created At'].replace("T", " ").replace(".000Z", "")
# alkaseeba_active_seller['Created At']= pd.to_datetime(alkaseeba_active_seller['Created At'],format="%Y-%m-%d")

# alkb_active_sellers = alkaseeba_active_seller.groupby(['Created At','SKU'],as_index=False)['Seller Mobile'].count()
# alkb_active_sellers.rename(columns = {'Seller Mobile':'Active Seller Count'}, inplace = True)
# alkb_active_sellers['Active Seller Count'].nunique
# alkb_active_sellers['Active Seller Count'].max()
# alkb_active_sellers

In [251]:
# pd.set_option('display.max_rows', 50)
# #adding active seller count based on date and sku
# main_dataset= pd.merge(main_dataset,alkb_active_sellers, left_on=['Uniform_Date','sku_code'],right_on=['Created At','SKU'], how = 'left',suffixes = ("","_"))
# main_dataset

### Add for Active Customers

In [252]:
df_merged_3.columns

Index(['Uniform_Date', 'order_type', 'order_id', 'order_code', 'sku_code',
       'sku_name', 'merchant', 'merchant_name', 'merchant_type', 'industry',
       'sku_pieces', 'items_cost', 'cash_returned', 'last_status', 'in_stock',
       'order_date', 'created_at', 'created_date', 'fulfilled_on',
       'fulfilled_date', 'modified_at', 'delivered_date', 'area',
       'mapped_area', 'valid_area', 'building_no', 'city', 'mapped_city',
       'zone_name', 'valid_city', 'valid_country', 'country', 'location_id',
       'invoice_id', 'valid_phone', 'address_line1', 'valid_address_line1',
       'is_work_address', 'user_id_main', 'customer_name', 'phone_number_main',
       'courier_name', 'tracking_number', 'week_number'],
      dtype='object')

In [253]:
active_customer =df_merged_3.copy()

active_customer['Created At']= pd.to_datetime(df_merged_3['created_at'],format="%Y-%m-%d") 


active_customers = active_customer.groupby(['Created At','sku_code'],as_index=False)['customer_name'].count()
active_customers.rename(columns = {'customer_name':'Active Customers Count'}, inplace = True)
active_customers['Active Customers Count'].nunique
active_customers

,Created At,sku_code,Active Customers Count
0,2021-02-08,31RL9YTE,1
1,2021-02-08,3H6F4MB2,1
2,2021-02-08,515KBA27,1
3,2021-02-08,CMH001,1
4,2021-02-08,FIHBXSW,1
...,...,...,...
254124,2023-02-07,موديل نيللي ابيض XXXL,3
254125,2023-02-07,موديل نيللي اسود L,4
254126,2023-02-07,موديل نيللي اسود XXL,4
254127,2023-02-07,موديل نيللي جنزاري XXL,2


In [254]:
pd.set_option('display.max_rows', 50)
#adding active seller count based on date and sku
main_dataset= pd.merge(main_dataset,active_customers, left_on=['Uniform_Date','sku_code'],right_on=['Created At','sku_code'], how = 'left',suffixes = ("","_"))
main_dataset

,Uniform_Date,Join_Dates,order_code,order_date,sku_code,sku_pieces,merchant_type,merchant,customer_name,industry,week_number,items_cost,in_stock,inv_on_hand,inv_in_transit,Year,month,Total_Inv,Created At,Active Customers Count
0,2021-02-08,2021-02-08,FIF-OT-FIF-1263,2021-02-08,515KBA27,1.0,Brand,fif,Ahmed Talaat,Fashion,2021-06,515.0,NaN,0.0,0.0,2021.0,2.0,0.0,2021-02-08,1.0
1,2021-02-08,2021-02-08,FIF-OT-FIF-2000,2021-02-08,FIHBXSW,1.0,Brand,fif,Sarah Mohamed,Fashion,2021-06,0.0,NaN,0.0,0.0,2021.0,2.0,0.0,2021-02-08,1.0
2,2021-02-08,2021-02-08,FIF-OT-FIF-2000,2021-02-08,FISLWLG,1.0,Brand,fif,Sarah Mohamed,Fashion,2021-06,0.0,NaN,0.0,0.0,2021.0,2.0,0.0,2021-02-08,1.0
3,2021-02-08,2021-02-08,HIB-OT-401628,2021-02-08,CMH001,1.0,Brand,hibrew,David Azmy,Home,2021-06,0.0,NaN,0.0,0.0,2021.0,2.0,0.0,2021-02-08,1.0
4,2021-02-08,2021-02-08,FIF-OT-FIF-1252_1,2021-02-08,3H6F4MB2,1.0,Brand,fif,Omar Mahsoub,Fashion,2021-06,0.0,NaN,0.0,0.0,2021.0,2.0,0.0,2021-02-08,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1024896,2023-02-07,2023-02-07,BND-OT-6280,2023-02-07,10131038,1.0,Brand,Beindie,Emmy Tarek,Fashion,2023-06,1300.0,True,0.0,0.0,2023.0,2.0,0.0,2023-02-07,4.0
1024897,2023-02-07,2023-02-07,ZHO-OT-66148,2023-02-07,101,1.0,Brand,Zohor,NaN,Trendy Products,2023-06,345.0,True,17479.0,0.0,2023.0,2.0,17479.0,2023-02-07,2.0
1024898,2023-02-07,2023-02-07,MDK-OT-9876937GCM,2023-02-07,em-0088,1.0,Seller,Mo3edatk,نادين توفيق,Marketplace,2023-06,360.0,True,0.0,0.0,2023.0,2.0,0.0,2023-02-07,6.0
1024899,2023-02-07,2023-02-07,SHC-OT-12485,2023-02-07,MB049201,1.0,Brand,SohatiCare,Mary Wageh,Cosmetics,2023-06,593.1,True,0.0,0.0,2023.0,2.0,0.0,2023-02-07,3.0


In [255]:
pd.set_option('display.max_rows', 50)
# main_dataset.dropna(subset = main_dataset.columns[main_dataset.columns != 'Uniform_Date'], axis =0, how = 'all',inplace = True)
# main_dataset.columns[main_dataset.columns != 'Uniform_Date']
main_dataset.isna().sum()


Uniform_Date                  0
Join_Dates                    1
order_code                    1
order_date                    1
sku_code                      1
sku_pieces                    1
merchant_type                 1
merchant                      1
customer_name             29757
industry                      1
week_number                   1
items_cost                    1
in_stock                  50188
inv_on_hand                   1
inv_in_transit                1
Year                          1
month                         1
Total_Inv                     1
Created At                    1
Active Customers Count        1
dtype: int64

In [256]:
#filling na for active sellers with zero for (testing only)
main_dataset['Active Customers Count'] = main_dataset['Active Customers Count'].fillna(0)
main_dataset.isna().sum()

Uniform_Date                  0
Join_Dates                    1
order_code                    1
order_date                    1
sku_code                      1
sku_pieces                    1
merchant_type                 1
merchant                      1
customer_name             29757
industry                      1
week_number                   1
items_cost                    1
in_stock                  50188
inv_on_hand                   1
inv_in_transit                1
Year                          1
month                         1
Total_Inv                     1
Created At                    1
Active Customers Count        0
dtype: int64

### Add for SR (Delivery Rate normalized columns for better results) 

In [257]:
sku_dr

,Join_Dates,sku_code,created_at,delivered,fulfilled,in transit,lost,on hold,pending,pick up,processing,ready,returned to origin
0,2021-02-08,31RL9YTE,2021-02-08,1.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0
1,2021-02-08,3H6F4MB2,2021-02-08,1.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0
2,2021-02-08,515KBA27,2021-02-08,1.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0
3,2021-02-08,CMH001,2021-02-08,0.0,0.0,0.00,0.0,0.0,0.0,1.0,0.00,0.0,0.0
4,2021-02-08,FIHBXSW,2021-02-08,1.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
254125,2023-02-07,موديل نيللي ابيض XXXL,2023-02-07,0.0,0.0,1.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0
254126,2023-02-07,موديل نيللي اسود L,2023-02-07,0.0,0.0,0.75,0.0,0.0,0.0,0.0,0.25,0.0,0.0
254127,2023-02-07,موديل نيللي اسود XXL,2023-02-07,0.0,0.0,0.75,0.0,0.0,0.0,0.0,0.25,0.0,0.0
254128,2023-02-07,موديل نيللي جنزاري XXL,2023-02-07,0.0,0.0,0.50,0.0,0.0,0.0,0.0,0.50,0.0,0.0


In [258]:
#adding active seller count based on date and sku
main_dataset= pd.merge(main_dataset,sku_dr, left_on=['Uniform_Date','sku_code'],right_on=['Join_Dates','sku_code'], how = 'left',suffixes = ("","_dr"))
main_dataset

,Uniform_Date,Join_Dates,order_code,order_date,sku_code,sku_pieces,merchant_type,merchant,customer_name,industry,week_number,items_cost,in_stock,inv_on_hand,inv_in_transit,Year,month,Total_Inv,Created At,Active Customers Count,Join_Dates_dr,created_at,delivered,fulfilled,in transit,lost,on hold,pending,pick up,processing,ready,returned to origin
0,2021-02-08,2021-02-08,FIF-OT-FIF-1263,2021-02-08,515KBA27,1.0,Brand,fif,Ahmed Talaat,Fashion,2021-06,515.0,NaN,0.0,0.0,2021.0,2.0,0.0,2021-02-08,1.0,2021-02-08,2021-02-08,1.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
1,2021-02-08,2021-02-08,FIF-OT-FIF-2000,2021-02-08,FIHBXSW,1.0,Brand,fif,Sarah Mohamed,Fashion,2021-06,0.0,NaN,0.0,0.0,2021.0,2.0,0.0,2021-02-08,1.0,2021-02-08,2021-02-08,1.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
2,2021-02-08,2021-02-08,FIF-OT-FIF-2000,2021-02-08,FISLWLG,1.0,Brand,fif,Sarah Mohamed,Fashion,2021-06,0.0,NaN,0.0,0.0,2021.0,2.0,0.0,2021-02-08,1.0,2021-02-08,2021-02-08,1.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
3,2021-02-08,2021-02-08,HIB-OT-401628,2021-02-08,CMH001,1.0,Brand,hibrew,David Azmy,Home,2021-06,0.0,NaN,0.0,0.0,2021.0,2.0,0.0,2021-02-08,1.0,2021-02-08,2021-02-08,0.0,0.0,0.000000,0.0,0.000000,0.000000,1.0,0.000000,0.000000,0.0
4,2021-02-08,2021-02-08,FIF-OT-FIF-1252_1,2021-02-08,3H6F4MB2,1.0,Brand,fif,Omar Mahsoub,Fashion,2021-06,0.0,NaN,0.0,0.0,2021.0,2.0,0.0,2021-02-08,1.0,2021-02-08,2021-02-08,1.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1024896,2023-02-07,2023-02-07,BND-OT-6280,2023-02-07,10131038,1.0,Brand,Beindie,Emmy Tarek,Fashion,2023-06,1300.0,True,0.0,0.0,2023.0,2.0,0.0,2023-02-07,4.0,2023-02-07,2023-02-07,0.0,0.0,0.250000,0.0,0.000000,0.250000,0.0,0.500000,0.000000,0.0
1024897,2023-02-07,2023-02-07,ZHO-OT-66148,2023-02-07,101,1.0,Brand,Zohor,NaN,Trendy Products,2023-06,345.0,True,17479.0,0.0,2023.0,2.0,17479.0,2023-02-07,2.0,2023-02-07,2023-02-07,0.0,0.0,0.444444,0.0,0.000000,0.000000,0.0,0.185185,0.370370,0.0
1024898,2023-02-07,2023-02-07,MDK-OT-9876937GCM,2023-02-07,em-0088,1.0,Seller,Mo3edatk,نادين توفيق,Marketplace,2023-06,360.0,True,0.0,0.0,2023.0,2.0,0.0,2023-02-07,6.0,2023-02-07,2023-02-07,0.0,0.0,0.000000,0.0,0.000000,0.166667,0.0,0.000000,0.833333,0.0
1024899,2023-02-07,2023-02-07,SHC-OT-12485,2023-02-07,MB049201,1.0,Brand,SohatiCare,Mary Wageh,Cosmetics,2023-06,593.1,True,0.0,0.0,2023.0,2.0,0.0,2023-02-07,3.0,2023-02-07,2023-02-07,0.0,0.0,0.000000,0.0,0.333333,0.333333,0.0,0.333333,0.000000,0.0


In [259]:
main_dataset.dtypes

Uniform_Date              datetime64[ns]
Join_Dates                datetime64[ns]
order_code                        object
order_date                datetime64[ns]
sku_code                          object
sku_pieces                       float64
merchant_type                     object
merchant                          object
customer_name                     object
industry                          object
week_number                       object
items_cost                       float64
in_stock                          object
inv_on_hand                      float64
inv_in_transit                   float64
Year                             float64
month                            float64
Total_Inv                        float64
Created At                datetime64[ns]
Active Customers Count           float64
Join_Dates_dr             datetime64[ns]
created_at                datetime64[ns]
delivered                        float64
fulfilled                        float64
in transit      

In [260]:
main_dataset

,Uniform_Date,Join_Dates,order_code,order_date,sku_code,sku_pieces,merchant_type,merchant,customer_name,industry,week_number,items_cost,in_stock,inv_on_hand,inv_in_transit,Year,month,Total_Inv,Created At,Active Customers Count,Join_Dates_dr,created_at,delivered,fulfilled,in transit,lost,on hold,pending,pick up,processing,ready,returned to origin
0,2021-02-08,2021-02-08,FIF-OT-FIF-1263,2021-02-08,515KBA27,1.0,Brand,fif,Ahmed Talaat,Fashion,2021-06,515.0,NaN,0.0,0.0,2021.0,2.0,0.0,2021-02-08,1.0,2021-02-08,2021-02-08,1.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
1,2021-02-08,2021-02-08,FIF-OT-FIF-2000,2021-02-08,FIHBXSW,1.0,Brand,fif,Sarah Mohamed,Fashion,2021-06,0.0,NaN,0.0,0.0,2021.0,2.0,0.0,2021-02-08,1.0,2021-02-08,2021-02-08,1.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
2,2021-02-08,2021-02-08,FIF-OT-FIF-2000,2021-02-08,FISLWLG,1.0,Brand,fif,Sarah Mohamed,Fashion,2021-06,0.0,NaN,0.0,0.0,2021.0,2.0,0.0,2021-02-08,1.0,2021-02-08,2021-02-08,1.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
3,2021-02-08,2021-02-08,HIB-OT-401628,2021-02-08,CMH001,1.0,Brand,hibrew,David Azmy,Home,2021-06,0.0,NaN,0.0,0.0,2021.0,2.0,0.0,2021-02-08,1.0,2021-02-08,2021-02-08,0.0,0.0,0.000000,0.0,0.000000,0.000000,1.0,0.000000,0.000000,0.0
4,2021-02-08,2021-02-08,FIF-OT-FIF-1252_1,2021-02-08,3H6F4MB2,1.0,Brand,fif,Omar Mahsoub,Fashion,2021-06,0.0,NaN,0.0,0.0,2021.0,2.0,0.0,2021-02-08,1.0,2021-02-08,2021-02-08,1.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1024896,2023-02-07,2023-02-07,BND-OT-6280,2023-02-07,10131038,1.0,Brand,Beindie,Emmy Tarek,Fashion,2023-06,1300.0,True,0.0,0.0,2023.0,2.0,0.0,2023-02-07,4.0,2023-02-07,2023-02-07,0.0,0.0,0.250000,0.0,0.000000,0.250000,0.0,0.500000,0.000000,0.0
1024897,2023-02-07,2023-02-07,ZHO-OT-66148,2023-02-07,101,1.0,Brand,Zohor,NaN,Trendy Products,2023-06,345.0,True,17479.0,0.0,2023.0,2.0,17479.0,2023-02-07,2.0,2023-02-07,2023-02-07,0.0,0.0,0.444444,0.0,0.000000,0.000000,0.0,0.185185,0.370370,0.0
1024898,2023-02-07,2023-02-07,MDK-OT-9876937GCM,2023-02-07,em-0088,1.0,Seller,Mo3edatk,نادين توفيق,Marketplace,2023-06,360.0,True,0.0,0.0,2023.0,2.0,0.0,2023-02-07,6.0,2023-02-07,2023-02-07,0.0,0.0,0.000000,0.0,0.000000,0.166667,0.0,0.000000,0.833333,0.0
1024899,2023-02-07,2023-02-07,SHC-OT-12485,2023-02-07,MB049201,1.0,Brand,SohatiCare,Mary Wageh,Cosmetics,2023-06,593.1,True,0.0,0.0,2023.0,2.0,0.0,2023-02-07,3.0,2023-02-07,2023-02-07,0.0,0.0,0.000000,0.0,0.333333,0.333333,0.0,0.333333,0.000000,0.0


### Adding Customer Confirmation Rate

In [261]:
Customer_confirmation_rate = df_merged_3.copy()
Customer_confirmation_rate

,Uniform_Date,order_type,order_id,order_code,sku_code,sku_name,merchant,merchant_name,merchant_type,industry,sku_pieces,items_cost,cash_returned,last_status,in_stock,order_date,created_at,created_date,fulfilled_on,fulfilled_date,modified_at,delivered_date,area,mapped_area,valid_area,building_no,city,mapped_city,zone_name,valid_city,valid_country,country,location_id,invoice_id,valid_phone,address_line1,valid_address_line1,is_work_address,user_id_main,customer_name,phone_number_main,courier_name,tracking_number,week_number
0,2021-02-08,outbound_order,fad5f9a2-30a2-4eda-980d-8dc6de6fe3e4,FIF-OT-FIF-1263,515KBA27,Basic Unisex Charcoal Hoodie - XL,fif,Fouad (fif),Brand,Fashion,1.0,515.0,0.0,delivered,NaN,2021-02-08,2021-02-08,2021-02-08,2021-02-09 00:00:00+00:00,2021-02-09,2021-02-08 11:42:11.249284+00:00,2021-02-10,Zamalek,Zamalek,True,4,Cairo,Cairo,cairo_giza,True,True,Egypt,113.0,11843.0,True,4 omaret al yamani Street Ground floor,True,False,1001.0,Ahmed Talaat,01001611604,Loadbugs,140981,2021-06
1,2021-02-08,outbound_order,63496d77-86d1-4345-8a46-4400a0f21a38,FIF-OT-FIF-2000,FIHBXSW,Beirut Hoodie-XS-White,fif,Fouad (fif),Brand,Fashion,1.0,0.0,0.0,delivered,NaN,2021-02-08,2021-02-08,2021-02-08,2021-02-09 00:00:00+00:00,2021-02-09,2021-02-08 11:50:42.125285+00:00,2021-02-10,Nasr City,Nasr City,True,24,Cairo,Cairo,cairo_giza,True,True,Egypt,104.0,11843.0,True,24 Ahmed Fakhry Street,True,False,1001.0,Sarah Mohamed,01286590341,Loadbugs,140982,2021-06
2,2021-02-08,outbound_order,63496d77-86d1-4345-8a46-4400a0f21a38,FIF-OT-FIF-2000,FISLWLG,Lil Wayne Sweater-L-Grey,fif,Fouad (fif),Brand,Fashion,1.0,0.0,0.0,delivered,NaN,2021-02-08,2021-02-08,2021-02-08,2021-02-09 00:00:00+00:00,2021-02-09,2021-02-08 11:50:42.125285+00:00,2021-02-10,Nasr City,Nasr City,True,24,Cairo,Cairo,cairo_giza,True,True,Egypt,104.0,11843.0,True,24 Ahmed Fakhry Street,True,False,1001.0,Sarah Mohamed,01286590341,Loadbugs,140982,2021-06
3,2021-02-08,outbound_order,b53b99fe-a36e-47be-a5b9-ed3e91109e2a,HIB-OT-401628,CMH001,HiBrew Coffee Machine,hibrew,HiBREW (hibrew),Brand,Home,1.0,0.0,0.0,pick up,NaN,2021-02-08,2021-02-08,2021-02-08,2021-02-08 00:00:00+00:00,2021-02-08,2021-07-07 16:17:43.511658+00:00,NaN,3rd Settlement,NaN,False,Warehouse Pickup,Cairo,Cairo,cairo_giza,True,True,Egypt,NaN,11863.0,True,Warehouse Pickup,False,False,1004.0,David Azmy,01234567890,Flextock (Same day),Pick up,2021-06
4,2021-02-08,outbound_order,05e3f750-4fa1-4c9e-8b78-1f15d630af0e,FIF-OT-FIF-1252_1,3H6F4MB2,FIF Basic Unisex Black On Black Sweatpants Gre...,fif,Fouad (fif),Brand,Fashion,1.0,0.0,0.0,delivered,NaN,2021-02-08,2021-02-08,2021-02-08,2021-02-10 00:00:00+00:00,2021-02-10,2021-02-08 13:24:56.982701+00:00,2021-02-11,Heliopolis,Heliopolis,True,97,Cairo,Cairo,cairo_giza,True,True,Egypt,99.0,11843.0,True,97 elmarghany street masr el gdeda,True,False,1001.0,Omar Mahsoub,01005332092,Loadbugs,141086,2021-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1024896,2023-02-07,outbound_order,f3824c48-7e1c-4b49-95dd-7429bdfc66c0,BND-OT-6280,10131038,Be-Strong Blue Tie-dye - 38,Beindie,Farah (Beindie),Brand,Fashion,1.0,1300.0,0.0,pending,True,2023-02-07,2023-02-07,2023-02-07,NaN,NaN,2023-02-07 22:27:36.048062+00:00,NaN,New Cairo,New Cairo,True,0,Cairo,Cairo,cairo_giza,True,True,Egypt,105.0,NaN,True,Compound Lakeview,True,False,1628.0,Emmy Tarek,01018848473,NaN,NaN,2023-06
1024897,2023-02-07,outbound_order,205151a1-7cb6-4bc6-b0b1-cdb188240501,ZHO-OT-66148,101,السبيرولينا,Zohor,Mohamed Marzok(Zohor),Brand,Trendy Products,1.0,345.0,0.0,processing,True,2023-02-07,2023-02-07,2023-02-07,NaN,NaN,2023-02-07 10:10:35.900169+00:00,NaN,الجيزة,Agouza,True,1,الهرم,Giza,cairo_giza,True,True,Egypt,168.0,NaN,True,محافظة الجيزة - الهرم - ٨٣ شارع الهرم - فوق نف...,True,False,1269.0,NaN,01007912692,JT Express,JEG000033457762,2023-06
1024898,2023-02-07,outbound_order,63e0eb01-105e-4fb4

In [262]:
Customer_confirmation_rate.last_status.unique

<bound method Series.unique of 0           delivered
1           delivered
2           delivered
3             pick up
4           delivered
              ...    
1024896       pending
1024897    processing
1024898         ready
1024899    processing
1024900    in transit
Name: last_status, Length: 1024901, dtype: object>

In [263]:
# view_orders_data = pd.read_csv("gs://flextock_ds_buuket/Demand Forecasting/TestData/Kaseeba_orders.csv")
#Function for getting seller confirmation rate df
def get_customer_confirm_rate(orders_data):
    
    def add_date_time_features(df: pd.DataFrame, date_feature: str, freq: str, drop=True) -> pd.DataFrame:

        df['Year'] = pd.to_datetime(df[date_feature]).dt.year

        if freq.lower() == 'week':
            df[freq] = pd.to_datetime(df[date_feature]).dt.isocalendar().week
        elif freq.lower() == 'month':
            df[freq] = pd.to_datetime(df[date_feature]).dt.month

        if drop:
            df.drop(date_feature, axis=1, inplace=True)

    def subtract_dates(arg_1, arg_2, **kwrgs) -> pd.Series:
        return (pd.to_datetime(arg_1) - pd.to_datetime(arg_2, **kwrgs)).dt.days

    freq = 'month'
    view_orders_data = orders_data.copy()
    view_orders_data['created_at'] = pd.to_datetime(view_orders_data['created_at']).dt.date
    add_date_time_features(view_orders_data, 'created_at', freq, drop=False)

    aggfunc_for_features = {'last_status': 'count'}

    df_pivot = pd.pivot_table(view_orders_data, index=['Year', freq, 'customer_name'], columns=['last_status'], fill_value=0, aggfunc=aggfunc_for_features)
    df_pivot.reset_index(inplace=True)
    # df_pivot.reset_index(inplace=True)
    df_pivot.head()

    
    num = ['delivered', 'returned to origin', 'lost','fulfilled']
    denom = ['pick up', 'in transit', 'processing', 'pending', 'on hold', 'ready']
    
    # Calculating CR
    df_pivot['Confirmation_Rate'] = df_pivot.last_status[num].sum(axis=1) / df_pivot.last_status[denom].sum(axis=1)

    df_pivot = df_pivot[['Year', 'month', 'customer_name', 'Confirmation_Rate']]
    # df_pivot.rename(columns={'customer_name_': 'Phone Number', 'Year': 'Year', 'month_': 'month'}, inplace=True)
    df_pivot.columns =  df_pivot.columns.droplevel(1)
    df_pivot.head()
    
    return df_pivot

In [264]:
#Customer confirmation rate call
customer_confirm_rate = get_customer_confirm_rate(Customer_confirmation_rate)
customer_confirm_rate

,Year,month,customer_name,Confirmation_Rate
0,2021.0,2.0,7ota O,0.0
1,2021.0,2.0,A Bed,0.0
2,2021.0,2.0,AA Abdelfatah,0.0
3,2021.0,2.0,Abdalrahman Thamer,inf
4,2021.0,2.0,Abdelrahman Hany,inf
...,...,...,...,...
450367,2023.0,2.0,‪Nahla Kamal‬‏,0.0
450368,2023.0,2.0,‪Noor Fawzy‬‏,0.0
450369,2023.0,2.0,‪Reem Mamdouh,0.0
450370,2023.0,2.0,‪mohamed elswefy‬‏,0.0


In [265]:
main_dataset

,Uniform_Date,Join_Dates,order_code,order_date,sku_code,sku_pieces,merchant_type,merchant,customer_name,industry,week_number,items_cost,in_stock,inv_on_hand,inv_in_transit,Year,month,Total_Inv,Created At,Active Customers Count,Join_Dates_dr,created_at,delivered,fulfilled,in transit,lost,on hold,pending,pick up,processing,ready,returned to origin
0,2021-02-08,2021-02-08,FIF-OT-FIF-1263,2021-02-08,515KBA27,1.0,Brand,fif,Ahmed Talaat,Fashion,2021-06,515.0,NaN,0.0,0.0,2021.0,2.0,0.0,2021-02-08,1.0,2021-02-08,2021-02-08,1.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
1,2021-02-08,2021-02-08,FIF-OT-FIF-2000,2021-02-08,FIHBXSW,1.0,Brand,fif,Sarah Mohamed,Fashion,2021-06,0.0,NaN,0.0,0.0,2021.0,2.0,0.0,2021-02-08,1.0,2021-02-08,2021-02-08,1.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
2,2021-02-08,2021-02-08,FIF-OT-FIF-2000,2021-02-08,FISLWLG,1.0,Brand,fif,Sarah Mohamed,Fashion,2021-06,0.0,NaN,0.0,0.0,2021.0,2.0,0.0,2021-02-08,1.0,2021-02-08,2021-02-08,1.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
3,2021-02-08,2021-02-08,HIB-OT-401628,2021-02-08,CMH001,1.0,Brand,hibrew,David Azmy,Home,2021-06,0.0,NaN,0.0,0.0,2021.0,2.0,0.0,2021-02-08,1.0,2021-02-08,2021-02-08,0.0,0.0,0.000000,0.0,0.000000,0.000000,1.0,0.000000,0.000000,0.0
4,2021-02-08,2021-02-08,FIF-OT-FIF-1252_1,2021-02-08,3H6F4MB2,1.0,Brand,fif,Omar Mahsoub,Fashion,2021-06,0.0,NaN,0.0,0.0,2021.0,2.0,0.0,2021-02-08,1.0,2021-02-08,2021-02-08,1.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1024896,2023-02-07,2023-02-07,BND-OT-6280,2023-02-07,10131038,1.0,Brand,Beindie,Emmy Tarek,Fashion,2023-06,1300.0,True,0.0,0.0,2023.0,2.0,0.0,2023-02-07,4.0,2023-02-07,2023-02-07,0.0,0.0,0.250000,0.0,0.000000,0.250000,0.0,0.500000,0.000000,0.0
1024897,2023-02-07,2023-02-07,ZHO-OT-66148,2023-02-07,101,1.0,Brand,Zohor,NaN,Trendy Products,2023-06,345.0,True,17479.0,0.0,2023.0,2.0,17479.0,2023-02-07,2.0,2023-02-07,2023-02-07,0.0,0.0,0.444444,0.0,0.000000,0.000000,0.0,0.185185,0.370370,0.0
1024898,2023-02-07,2023-02-07,MDK-OT-9876937GCM,2023-02-07,em-0088,1.0,Seller,Mo3edatk,نادين توفيق,Marketplace,2023-06,360.0,True,0.0,0.0,2023.0,2.0,0.0,2023-02-07,6.0,2023-02-07,2023-02-07,0.0,0.0,0.000000,0.0,0.000000,0.166667,0.0,0.000000,0.833333,0.0
1024899,2023-02-07,2023-02-07,SHC-OT-12485,2023-02-07,MB049201,1.0,Brand,SohatiCare,Mary Wageh,Cosmetics,2023-06,593.1,True,0.0,0.0,2023.0,2.0,0.0,2023-02-07,3.0,2023-02-07,2023-02-07,0.0,0.0,0.000000,0.0,0.333333,0.333333,0.0,0.333333,0.000000,0.0


In [266]:
#function for merge maindataset with seller confirmation rate
def merge_df(*dfs):
    # dataframes = list(df for df in dfs)
    dataframes = [*dfs]
    df_merged = reduce(lambda left,right: pd.merge(left,right,on=['month', 'Year', 'customer_name'], how='left'), dataframes)
    return df_merged


In [267]:
# Call for merge b/w main dataset and seller confirmation rate
main_dataset = merge_df(main_dataset,customer_confirm_rate) 
main_dataset.tail(30)

,Uniform_Date,Join_Dates,order_code,order_date,sku_code,sku_pieces,merchant_type,merchant,customer_name,industry,week_number,items_cost,in_stock,inv_on_hand,inv_in_transit,Year,month,Total_Inv,Created At,Active Customers Count,Join_Dates_dr,created_at,delivered,fulfilled,in transit,lost,on hold,pending,pick up,processing,ready,returned to origin,Confirmation_Rate
1024871,2023-02-07,2023-02-07,PHX-OT-75589,2023-02-07,QIIV743S,1.0,Brand,Pharmix,Lobna Omar,Cosmetics,2023-06,369.0,True,1524.0,12.0,2023.0,2.0,1536.0,2023-02-07,230.0,2023-02-07,2023-02-07,0.0,0.0,0.173913,0.0,0.147826,0.408696,0.0,0.121739,0.147826,0.0,0.0
1024872,2023-02-07,2023-02-07,NTV-OT-17610,2023-02-07,G854EC5J,1.0,Brand,natavis,Hajar Mohamed,Cosmetics,2023-06,0.0,True,0.0,0.0,2023.0,2.0,0.0,2023-02-07,2.0,2023-02-07,2023-02-07,0.0,0.0,0.000000,0.0,0.000000,1.000000,0.0,0.000000,0.000000,0.0,0.0
1024873,2023-02-07,2023-02-07,NTV-OT-17610,2023-02-07,G854EC5J,1.0,Brand,natavis,Hajar Mohamed,Cosmetics,2023-06,0.0,True,0.0,0.0,2023.0,2.0,0.0,2023-02-07,2.0,2023-02-07,2023-02-07,0.0,0.0,0.000000,0.0,0.000000,1.000000,0.0,0.000000,0.000000,0.0,0.0
1024874,2023-02-07,2023-02-07,FNC-OT-SHQ4072,2023-02-07,SOD3,1.0,Brand,Funnycouples,Mohammed El shazly,Games,2023-06,335.0,True,1218.0,10.0,2023.0,2.0,1228.0,2023-02-07,44.0,2023-02-07,2023-02-07,0.0,0.0,0.204545,0.0,0.045455,0.500000,0.0,0.113636,0.136364,0.0,0.0
1024875,2023-02-07,2023-02-07,FNC-OT-SHQ4072,2023-02-07,SOD2,1.0,Brand,Funnycouples,Mohammed El shazly,Games,2023-06,335.0,True,1203.0,10.0,2023.0,2.0,1213.0,2023-02-07,44.0,2023-02-07,2023-02-07,0.0,0.0,0.204545,0.0,0.045455,0.500000,0.0,0.113636,0.136364,0.0,0.0
1024876,2023-02-07,2023-02-07,SOD-OT-FRRK2AE3120230206v01khms1011200544964,2023-02-07,RK2,3.0,Brand,Mindev,علي عبد الرازق العمراوي,Cosmetics,2023-06,860.0,True,1736.0,32.0,2023.0,2.0,1768.0,2023-02-07,74.0,2023-02-07,2023-02-07,0.0,0.0,0.000000,0.0,0.000000,1.000000,0.0,0.000000,0.000000,0.0,0.0
1024877,2023-02-07,2023-02-07,PHX-OT-75626,2023-02-07,QIIV743S,1.0,Brand,Pharmix,Dina mohammed,Cosmetics,2023-06,369.0,False,1524.0,12.0,2023.0,2.0,1536.0,2023-02-07,230.0,2023-02-07,2023-02-07,0.0,0.0,0.173913,0.0,0.147826,0.408696,0.0,0.121739,0.147826,0.0,0.0
1024878,2023-02-07,2023-02-07,PHX-OT-75626,2023-02-07,QIIV743S,1.0,Brand,Pharmix,Dina mohammed,Cosmetics,2023-06,369.0,False,1524.0,12.0,2023.0,2.0,1536.0,2023-02-07,230.0,2023-02-07,2023-02-07,0.0,0.0,0.173913,0.0,0.147826,0.408696,0.0,0.121739,0.147826,0.0,0.0
1024879,2023-02-07,2023-02-07,BCL-OT-10726,2023-02-07,6221198257890,1.0,Brand,TheBasicLook,Mariam Roshdy,Fashion,2023-06,365.0,True,0.0,0.0,2023.0,2.0,0.0,2023-02-07,4.0,2023-02-07,2023-02-07,0.0,0.0,0.250000,0.0,0.000000,0.250000,0.0,0.000000,0.500000,0.0,0.0
1024880,2023-02-07,2023-02-07,BCL-OT-10726,2023-02-07,6221198258583,1.0,Brand,TheBasicLook,Mariam Roshdy,Fashion,2023-06,365.0,True,220.0,8.0,2023.0,2.0,228.0,2023-02-07,3.0,2023-02-07,2023-02-07,0.0,0.0,0.333333,0.0,0.000000,0.333333,0.0,0.000000,0.333333,0.0,0.0


#### Adding for missing columns in maindataset dummy columns to cover for absent column due to absent data. Read below. Jan 2023

In [312]:
# Added Jan 2023
# Function to add missing columns with zeros so that training dataset and pre-processed dataset has same columms
#Introduced to primarily cater for transitory missing 'ready' columns at some time during mid-night in outbounds filtered for kaseeba
def add_col_to_df(df,*cols):
    for col in cols:
        if col not in df:
            print('Adding column with zeros')
            df[col] = 0


In [313]:
#Call add_col_to_df function to add ready column. Can add multiple columns
#Example
# add_col_to_df(df_test,'test','pass','not pass')
# df_test.head()

#Actual 
add_col_to_df(main_dataset,'ready')
main_dataset.head()
# df_test

Adding column with zeros


,Uniform_Date,Join_Dates,order_code,order_date,Seller Mobile,sku_code,sku_pieces,Product Category,Subcategory,Season,...,fulfilled,in transit,lost,on hold,pending,pick up,processing,returned to origin,Confirmation_Rate,ready
0,2021-12-13,2021-12-13,BAS-OT-105767,2021-12-13,1.200011e+09,MOB-NK,1.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.823529,0.980392,0
1,2021-12-13,2021-12-13,BAS-OT-918337,2021-12-13,1.200011e+09,MOB-NK,1.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.823529,0.980392,0
2,2021-12-13,2021-12-13,BAS-OT-275077,2021-12-13,1.200011e+09,SHEN-FAK,1.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.980392,0
3,2021-12-13,2021-12-13,BAS-OT-548669,2021-12-13,1.200011e+09,MOB-NK,1.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.823529,0.980392,0
4,2021-12-13,2021-12-13,BAS-OT-914580,2021-12-13,1.200011e+09,MOB-NK,1.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.823529,0.980392,0


## Main Dataset

In [701]:
#save to local pwd
main_dataset.to_excel('main_dataset_merchant.xlsx')
main_dataset.to_csv('main_dataset_merchant.csv')

### Write to GCS Bucket

In [4]:
#save to gcs bucket
import os
from google.cloud import storage
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'dstest-service-account-file.json'

In [11]:
# File path and name.
# file_path = r'/home/jupyter/'
file_path = os.getcwd() +'/'
fileName = 'main_dataset_merchant.xlsx'

full_path = file_path + fileName
print(full_path)

# uploading file to GCS
storage_client = storage.Client()
bucket = storage_client.get_bucket("flextock_ds_buuket")
blob_name = fileName
blob = bucket.blob("Demand Forecasting/pre-processed-files/" + blob_name)
# logger.debug(blob)

with open(full_path, 'rb') as f:
          blob.upload_from_file(f)
print("main_dataset exported to GCS successfully")
# logger.info("view_orders_data file exported to GCS successfully")

/home/jupyter/DF_datapipeline/main_dataset.xlsx
main_dataset exported to GCS successfully
